<h2>Computer Graphics and Augmented Reality</h2>
<h2>Computational Visualization</h2>

<h1>Racing Track Scenario</h1>

<h3>Tânia Gonçalves</h3>

<p>
The present notebook includes the implementation and the report for the racing track scenario. Initially, all the provided classes are included, and the vertex and fragment shaders used are defined, including shaders for texture and lighting. All the images for the textures are located in the Images folder.
</p>

In [1]:
%%html    
<script src="http://is3l.isr.uc.pt/~pm/CGRA/JS/deecshader.js"></script>
<script src="http://is3l.isr.uc.pt/~pm/CGRA/JS/deecapp.js"></script> 
<script src="http://is3l.isr.uc.pt/~pm/CGRA/JS/cgraobject.js"></script>
<script src='https://git.io/glm-js.min.js'></script>
<script src='http://is3l.isr.uc.pt/~pm/CGRA/JS/cgratexture.js'></script>

In [2]:
%%html
<script id="my-vertex-shader" type="x-shader/x-vertex">
precision highp float;

attribute  vec3 in_Position;
attribute  vec3 in_Color;

varying  vec3 ex_Color;

uniform mat4 MVP;

void main(void) {
    // Set the W coordinate to 1
    gl_Position = MVP * vec4(in_Position, 1.0);

    ex_Color = in_Color;
}
</script>

In [3]:
%%html
<script id="my-fragment-shader" type="x-shader/x-fragment">
precision highp float;

varying  vec3 ex_Color;

void main(void) {
    // Pass through our original color with full opacity.
    gl_FragColor = vec4(ex_Color,1.0);
}
</script>

In [4]:
%%html
<script id="my-vertex-shaderT" type="x-shader/x-vertex">
precision mediump float;

attribute  vec3 in_Position;
attribute  vec3 in_Color;
attribute vec2 in_texcoords;
uniform mat4 MVP;

varying  vec3 ex_Color;
varying  vec2 vTextureCoord;

void main() {
  
    gl_Position = MVP * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);
    vTextureCoord = in_texcoords;
    ex_Color = in_Color;
}
</script>

In [5]:
%%html
<script id="my-fragment-shaderT" type="x-shader/x-fragment">
precision mediump float;
varying  vec2 vTextureCoord;
varying  vec3 ex_Color;

uniform sampler2D uSampler;

void main() {
     gl_FragColor = texture2D(uSampler, vTextureCoord);
}
</script>

In [6]:
%%html
<script id="my-vertex-shaderL" type="x-shader/x-vertex">
precision mediump float;

//# version 330 core - in -> attribute // out -> varying

attribute  vec3 in_Position; // attribute vec4 VertexPosition; // = in_Position
attribute  vec3 in_Color;    // attribute vec4 VertexColor; // = in_Color
attribute  vec2 in_texcoords;

varying  vec3 ex_Color;      // varying vec4 Color; // = ex_color
varying highp vec2 vTextureCoord;

uniform mat4 MVP;

uniform mat4 MVPMatrix;
uniform mat3 NormalMatrix; // to transform normals ,pre-perspective
attribute vec3 VertexNormal;

varying vec3 Normal;
varying vec4 Position; // "extra"


void main() {
    
    gl_Position = MVP * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);
    
    vTextureCoord = in_texcoords;
    
    ex_Color = in_Color; //Color = VertexColor;
    
    // we now need a surfacenormal
    // interpolate the normalized surface normal
    // transform the normal, without perspective, and normalize it
    Normal = normalize(NormalMatrix * VertexNormal);
    
    //gl_Position = MVPMatrix * VertexPosition;
    Position = MVPMatrix * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);    
    
    
}
</script>

In [7]:
%%html
<script id="my-fragment-shaderL" type="x-shader/x-fragment">
precision mediump float;
varying highp vec2 vTextureCoord;

uniform sampler2D uSampler;
varying  vec3 ex_Color; //attribute vec4 Color; // = ex_Color

uniform vec3 Ambient;
uniform vec3 LightColor;
uniform vec3 LightColor1;
uniform vec3 LightColor2;
uniform vec3 LightDirection; // direction toward the light
uniform vec3 LightDirection1; // direction toward the light
uniform vec3 LightDirection2; // direction toward the light

//uniform vec3 HalfVector; // surface orientation for shiniest spots

uniform float Shininess; // exponent for sharping highlights
uniform float Strength; // extra factor to adjust shininess

uniform vec3 ViewPosition; 
uniform vec3 ViewPosition1;
uniform vec3 ViewPosition2;


varying vec3 Normal; // surface normal, interpolate d between vertices
varying vec4 Position;


void main() {
    
    vec3 LightDirectionD = LightDirection - vec3(Position);
    LightDirectionD = LightDirectionD / length(LightDirectionD);
    
    vec3 LightDirectionD1 = LightDirection1 - vec3(Position);
    LightDirectionD1 = LightDirectionD1 / length(LightDirectionD1);
    
    vec3 LightDirectionD2 = LightDirection2 - vec3(Position);
    LightDirectionD2 = LightDirectionD2 / length(LightDirectionD2);
    
    vec3 ViewDirection = ViewPosition - vec3(Position);
    ViewDirection = ViewDirection / length(ViewDirection);
    
    vec3 ViewDirection1 = ViewPosition1 - vec3(Position);
    ViewDirection1 = ViewDirection1 / length(ViewDirection1);
    
    vec3 ViewDirection2 = ViewPosition2 - vec3(Position);
    ViewDirection2 = ViewDirection2 / length(ViewDirection2);
    
    
    vec3 HalfVector = normalize(LightDirectionD + ViewDirection);
    vec3 HalfVector1 = normalize(LightDirectionD1 + ViewDirection1);
    vec3 HalfVector2 = normalize(LightDirectionD2 + ViewDirection2);
    
    
    // compute cosine of the directions, using dot products,
    // to see how much light would be reflected
    float diffuse = max(0.0, dot(Normal, LightDirectionD));
    float specular = max (0.0, dot(Normal, HalfVector));
    
    float diffuse1 = max(0.0, dot(Normal, LightDirectionD1));
    float specular1 = max (0.0, dot(Normal, HalfVector1));
    
    float diffuse2 = max(0.0, dot(Normal, LightDirectionD2));
    float specular2 = max (0.0, dot(Normal, HalfVector2));

    // surfaces facing away from the light
    // won’t be lit by the directional light
    if(diffuse == 0.0)
        specular = 0.0;
    else
        specular = pow(specular, Shininess); // sharpen the highlight
    
    if(diffuse1 == 0.0)
        specular1 = 0.0;
    else
        specular1 = pow(specular1, Shininess); // sharpen the highlight
    
    if(diffuse2 == 0.0)
        specular2 = 0.0;
    else
        specular2 = pow(specular2, Shininess); // sharpen the highlight
    
    vec3 scatteredLight = Ambient + LightColor * diffuse + LightColor1 * diffuse1 + LightColor2 * diffuse2;
    vec3 reflectedLight = LightColor * specular *  Strength + LightColor1 * specular1 * Strength+ LightColor2 * specular2 * Strength;
    
    // don’t modulate the underlying color with reflected light,
    // only with scattered light
    vec3 rgb = min(ex_Color.rgb * scatteredLight + reflectedLight , vec3 (1.0));
    
    //gl_FragColor = vec4(rgb, ex_Color.a);
    
    gl_FragColor = texture2D(uSampler, vTextureCoord) * vec4(rgb, 1.0);
    
    
}
</script>

<h1>Basic Shapes</h1>

<h2>Cube:</h2>

<p>
All vertices were defined one by one, using two triangles for each face, with each face having a different color, totaling 36 vertices. This includes textures and normals pointing outward. Here, all components (uniform variables) that will be used by the shaders are defined.  
</p>

In [8]:
%%html
<script>
class cubo extends CGRAobject{
    constructor(glcontext, r, g, b, samecolor, screen){
        super(glcontext); // initialize the parent class
        
        //var texCoords = [];

        this.numvertices = 36;
        this.vertices =
            [ // FRENTE
              // abc
              -0.4, 0.4, 0.4,
              0.4, 0.4, 0.4,
              -0.4, -0.4, 0.4,
              // cbd
              -0.4, -0.4, 0.4,
              0.4, 0.4, 0.4,
              0.4, -0.4, 0.4,
              // BAIXO
              // cdh
              -0.4, -0.4, 0.4,
              0.4, -0.4, 0.4,
              0.4, -0.4, -0.4,
              // hcg 
              0.4, -0.4, -0.4,
              -0.4, -0.4, 0.4,
              -0.4, -0.4, -0.4,
              // ESQUERDA
              // ace
              -0.4, 0.4, 0.4,
              -0.4, -0.4, 0.4,
              -0.4, 0.4, -0.4,
              // ecg 
              -0.4, 0.4, -0.4,
              -0.4, -0.4, 0.4,
              -0.4, -0.4, -0.4,
              // TRÁS
              // efh
              -0.4, 0.4, -0.4,
              0.4, 0.4, -0.4,
              0.4, -0.4, -0.4,
              // heg 
              0.4, -0.4, -0.4,
              -0.4, 0.4, -0.4,
              -0.4, -0.4, -0.4,
              // DIREITA
              // bdh 
              0.4, 0.4, 0.4,
              0.4, -0.4, 0.4,
              0.4, -0.4, -0.4,
              // hbf 
              0.4, -0.4, -0.4,
              0.4, 0.4, 0.4,
              0.4, 0.4, -0.4,
              // CIMA
              // aef
              -0.4, 0.4, 0.4,
              -0.4, 0.4, -0.4,
              0.4, 0.4, -0.4,
              // fba
              0.4, 0.4, -0.4,
              0.4, 0.4, 0.4,
              -0.4, 0.4, 0.4
             
            //a: -0.4, 0.4, 0.4,
            //b: 0.4, 0.4, 0.4,
            //c: -0.4, -0.4, 0.4,
            //d: 0.4, -0.4, 0.4,
            //e: -0.4, 0.4, -0.4,
            //f: 0.4, 0.4, -0.4,
            //g: -0.4, -0.4, -0.4,
            //h: 0.4, -0.4, -0.4,
              
              ];
        
        this.colors = [
            // FRENTE
            0.0, 0.0, 1.0, 
            0.0, 0.0, 1.0,
            0.0, 0.0, 1.0,
            0.0, 0.0, 1.0,
            0.0, 0.0, 1.0,
            0.0, 0.0, 1.0,
            // BAIXO
            0.0, 1.0, 1.0,
            0.0, 1.0, 1.0,
            0.0, 1.0, 1.0,
            0.0, 1.0, 1.0,
            0.0, 1.0, 1.0,
            0.0, 1.0, 1.0,
            // ESQUERDA
            1.0, 0.0, 1.0,
            1.0, 0.0, 1.0, 
            1.0, 0.0, 1.0,
            1.0, 0.0, 1.0,
            1.0, 0.0, 1.0,
            1.0, 0.0, 1.0,
            // TRÁS
            1.0, 1.0, 0.0,
            1.0, 1.0, 0.0,
            1.0, 1.0, 0.0,
            1.0, 1.0, 0.0,
            1.0, 1.0, 0.0,
            1.0, 1.0, 0.0,
            // DIREITA
            0.0, 1.0, 0.0,
            0.0, 1.0, 0.0,
            0.0, 1.0, 0.0,
            0.0, 1.0, 0.0,
            0.0, 1.0, 0.0,
            0.0, 1.0, 0.0,
            // CIMA
            1.0, 0.0, 0.0,
            1.0, 0.0, 0.0,
            1.0, 0.0, 0.0,
            1.0, 0.0, 0.0,
            1.0, 0.0, 0.0,
            1.0, 0.0, 0.0];
        
        var texCoords =
            [ // FRENTE
              // abc
              0.0, 0.33,
              0.0, 0.66,
              0.25, 0.33,

              // cbd
              0.25, 0.33,
              0.0, 0.66,
              0.25, 0.66,

              // BAIXO
              // cdh
              0.25, 0.33,
              0.25, 0.66,
              0.5, 0.66,

              // hcg
              0.5, 0.66,
              0.25, 0.33,
              0.5, 0.33,

              // ESQUERDA
              // ace
              0.25, 0,
              0.25, 0.33,
              0.5, 0,

              // ecg 
              0.5, 0,
              0.25, 0.33,
              0.5, 0.33,

              // TRÁS
              // efh
              0.75, 0.33,
              0.75, 0.66,
              0.5, 0.66,

              // heg 
              0.5, 0.66,
              0.75, 0.33,
              0.5, 0.33,

              // DIREITA
              // bdh 
              0.25, 1,
              0.25, 0.66,
              0.5, 0.66,

              // hbf 
              0.5, 0.66,
              0.25, 1,
              0.5, 1,

              // CIMA
              // aef
              1, 0.33,
              0.75, 0.33,
              0.75, 0.66,

              // fba
              0.75, 0.66,
              1, 0.66,
              1, 0.33

             
            //a: -0.4, 0.4, 0.4,
            //b: 0.4, 0.4, 0.4,
            //c: -0.4, -0.4, 0.4,
            //d: 0.4, -0.4, 0.4,
            //e: -0.4, 0.4, -0.4,
            //f: 0.4, 0.4, -0.4,
            //g: -0.4, -0.4, -0.4,
            //h: 0.4, -0.4, -0.4,
              
              ];
        
        var normais = [
          // FRENTE
          0.0, 0.0, 1.0,
          0.0, 0.0, 1.0,
          0.0, 0.0, 1.0,
          0.0, 0.0, 1.0,
          0.0, 0.0, 1.0,
          0.0, 0.0, 1.0,
            
          // BAIXO
          0.0, -1.0, 0.0,
          0.0, -1.0, 0.0,
          0.0, -1.0, 0.0,
          0.0, -1.0, 0.0,
          0.0, -1.0, 0.0,
          0.0, -1.0, 0.0,
            
          // ESQUERDA
          -1.0, 0.0, 0.0,
          -1.0, 0.0, 0.0,
          -1.0, 0.0, 0.0,
          -1.0, 0.0, 0.0,
          -1.0, 0.0, 0.0,
          -1.0, 0.0, 0.0,
            
          // TRÁS
          0.0, 0.0, -1.0,
          0.0, 0.0, -1.0,
          0.0, 0.0, -1.0,
          0.0, 0.0, -1.0,
          0.0, 0.0, -1.0,
          0.0, 0.0, -1.0,
            
          // DIREITA
          1.0, 0.0, 0.0,
          1.0, 0.0, 0.0,
          1.0, 0.0, 0.0,
          1.0, 0.0, 0.0,
          1.0, 0.0, 0.0,
          1.0, 0.0, 0.0,
 
          // CIMA
          0.0, 1.0, 0.0,
          0.0, 1.0, 0.0,
          0.0, 1.0, 0.0,
          0.0, 1.0, 0.0,
          0.0, 1.0, 0.0,
          0.0, 1.0, 0.0

        ]; 
        

        if(samecolor == 1){
            var newColor = [];
            for(var i = 0; i < this.numvertices; i++){
                newColor.push(r);
                newColor.push(g);
                newColor.push(b);
            }
            this.colors = newColor;
        }
        
        if(screen == 1){
            var newtexCoords = [];
            for(var i = 0; i < this.numvertices/6; i++){
                newtexCoords.push(0);
                newtexCoords.push(1);
                
                newtexCoords.push(1);
                newtexCoords.push(1);
                
                newtexCoords.push(0);
                newtexCoords.push(0);
                
                newtexCoords.push(0);
                newtexCoords.push(0);
                
                newtexCoords.push(1);
                newtexCoords.push(1);
                
                newtexCoords.push(1);
                newtexCoords.push(0);
            }
            texCoords = newtexCoords;
        }
        
        
        
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        var a = new Float32Array(this.vertices);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
        
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer); 
        var a = new Float32Array(this.colors);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
        
        this.texcoordsbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordsbuffer); 
        var a = new Float32Array(texCoords);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW); 
        
        this.normalbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.normalbuffer);    
        var a = new Float32Array(normais);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
    
        
    }  
}

class cuboL extends cubo{
    
    settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
    
    drawit(viewMat, projectionMat, NoiteDia, lamp1, lamp2, lightL1, lightL2){
        this.shaderprog.startUsing();
        
        // TEXTURAS
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordsbuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 0);
        
        
        // NORMAIS
        this.normalvertex = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "VertexNormal");
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.normalbuffer);
        
        this.gl.vertexAttribPointer(this.normalvertex, // Attribute location
                           3, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           this.gl.FALSE,  // 
                           3*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the beginint of a singles vertex to this attribute

        this.gl.enableVertexAttribArray(this.normalvertex);
        
        var NormalMat = glm.mat3(this.modelMat);
        
        var MVPMatrix = this.modelMat;
        //console.log('matrix:'+MVPMatrix);                                                                       

        var ambiente = glm.vec3(0.9, 0.9, 0.9);
        var light = glm.vec3(0.0, 0.0, 0.0);
        var light1 = glm.vec3(0.0, 0.0, 0.0);
        var light2 = glm.vec3(0.0, 0.0, 0.0);                                                                     
                                                                             
        var posicao = glm.vec3(0.0, 50.0, 0.0);
        var posicao1 = glm.vec3(0.0, 4.0, -30.0);
        var posicao2 = glm.vec3(0.0, 4.0, 30.0);
                                                                             
        var shine = 1.0;                                                                     
        var streng = 100.5;
        var camera = glm.vec3(0.0, 0.0, 0.0);
        var camera1 = glm.vec3(-150.0, -150.0, -50.0);
        var camera2 = glm.vec3(-150.0, -150.0, 50.0);
        
        
        posicao1 = glm.vec3(0.0, 4.0, -30);
        posicao2 = glm.vec3(0.0, 4.0, 30.0);
        camera1 = glm.vec3(-150, -150, -50.0);
        camera2 = glm.vec3(-150, -150, 50.0);
                                                    
        // Dia
        if(NoiteDia == 1){
            ambiente = glm.vec3(0.9, 0.9, 0.9);
            light =  glm.vec3(1.0, 1.0, 0.8);
            shine = 1.0;
            streng = 100.5;
        }
        
        //Noite
        if(NoiteDia == 0 && lamp1 == 1 && lamp2 == 1){
            ambiente = glm.vec3(0.25, 0.25, 0.25);
            light = glm.vec3(0.1, 0.1, 0.1);
            light1 = lightL1;
            light2 = lightL2;
            shine = 1.0;
            streng = 10.5;

        }else if(NoiteDia == 0 && lamp1 == 1 && lamp2 == 0){
            ambiente = glm.vec3(0.25, 0.25, 0.25);
            light = glm.vec3(0.1, 0.1, 0.1);
            light1 = lightL1;
            shine = 1.0;
            streng = 10.5;

        }else if(NoiteDia == 0 && lamp1 == 0 && lamp2 == 1){
            ambiente = glm.vec3(0.25, 0.25, 0.25);
            light = glm.vec3(0.1, 0.1, 0.1);
            light2 = lightL2;
            shine = 1.0;
            streng = 10.5;

        }else if(NoiteDia == 0 && lamp1 == 0 && lamp2 == 0){
            ambiente = glm.vec3(0.25, 0.25, 0.25);
            light = glm.vec3(0.1, 0.1, 0.1); 
            shine = 1.0;
            streng = 10.5;
        }
                                                                                                                   
        var matriz_normal = this.gl.getUniformLocation(this.shaderprog.shaderProgram,  "NormalMatrix");
        this.gl.uniformMatrix3fv(matriz_normal, false, NormalMat.array);

        var mvloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "MVPMatrix");
        this.gl.uniformMatrix4fv(mvloc,false, MVPMatrix.array);                                                                     
        
        var ambiente1 = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "Ambient");
        this.gl.uniform3fv(ambiente1, ambiente.array);
        
        var light_color = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "LightColor");
        this.gl.uniform3fv(light_color, light.array);
                                                                             
        var light_color1 = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "LightColor1");
        this.gl.uniform3fv(light_color1, light1.array);                                                                     
                                                                             
        var light_color2 = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "LightColor2");
        this.gl.uniform3fv(light_color2, light2.array);                                                                     
        
        var shininess = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "Shininess");
        this.gl.uniform1f(shininess, shine);
        
        var strength = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "Strength");
        this.gl.uniform1f(strength, streng);
        
        var posic = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "LightDirection");
        this.gl.uniform3fv(posic, posicao.array);
                                                                             
        var posic1 = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "LightDirection1");
        this.gl.uniform3fv(posic1, posicao1.array);
                                                                             
        var posic2 = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "LightDirection2");
        this.gl.uniform3fv(posic2, posicao2.array);
        
        var ViewPos = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "ViewPosition");
        this.gl.uniform3fv(ViewPos, camera.array);
        
        var ViewPos1 = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "ViewPosition1");
        this.gl.uniform3fv(ViewPos1, camera1.array);
        
        var ViewPos2 = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "ViewPosition2");
        this.gl.uniform3fv(ViewPos2, camera2.array);                                           
        
        
        // the parent method does the rest
        super.drawit(viewMat,projectionMat);
    }  
}
</script>

In [9]:
%%html
<canvas id="myCanvasCubo2" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp">

class myappCubo2 extends DEECapp{
    angRotacao = 0;
    
    initialize(){
          
        var fragsrc = document.getElementById("my-fragment-shaderL").text;
        var vertsrc = document.getElementById("my-vertex-shaderL").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        
        // perform other initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.67,0.923,0.99,1.0);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("Images/cuboparede.png");
        
        //r, g, b, samecolor, screen
        this.mycubo = new cuboL(this.gl, 0.9, 0.9, 0.9, 1, 0);
        this.mycubo.settexture(this.texture);
        this.mycubo.setShader(this.shaderprog);
        
    }
    
    render(){
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        
        this.angRotacao = this.angRotacao + 3;
        
        
        // TRANSLAÇÃO
        var TranslationMat = glm.translate(glm.mat4(1.0), glm.vec3(12.0,10.0,10.0));
        
        // ROTAÇÃO
        var RotMat = glm.rotate(glm.radians(this.angRotacao), glm.vec3(0.0,1.0,0.0));
        
        var ScalingMat = glm.scale(glm.vec3(3, 3, 3));

        var myModelMatrix = (TranslationMat['*'](ScalingMat)['*'](RotMat));
        
        // Projection and view matrices
        this.projM = glm.perspective(glm.radians(45),1,0.1,100);
        
        this.viewM = glm.lookAt(glm.vec3(1,6,8),glm.vec3(9.5,9.5,9.5),glm.vec3(0,1,0));


        this.mycubo.setModelTransformation(myModelMatrix);
        
        // Utilizar o método drawit fornecido
        this.mycubo.drawit(this.viewM, this.projM, 1, 0, 0);

    }
}

var appCubo2 = new myappCubo2('myCanvasCubo2');

appCubo2.run();
</script>

<h2>Cone:</h2>

<p>
    A loop was created, where for each iteration, three vertices are defined for the <strong>base triangle</strong> and another three vertices for the <strong>side triangle</strong>. The vertices for the base of the cone are defined according to a <strong>circle</strong> given by the coordinates (x, y, z) = (cos(ang), sin(ang), 0), where <em>ang</em> is an angle that multiplies by a variable <em>i</em> that increments in the loop, allowing the traversal from 0 to <strong>2π</strong>. The base of the cone, that is, the circle, has a variable <strong>ntriangles</strong> that indicates the number of triangles drawn in a <strong>complete revolution</strong>. Therefore, <em>ang = 2π/ntriangles</em>. This includes <strong>textures</strong> and <strong>normals</strong> pointing outward. Here, all components (<strong>uniform variables</strong>) that will be used by the shaders are defined.
</p>

In [10]:
%%html
<script>
class cone extends CGRAobject{ 
    constructor(glcontext, ntriangulos, r, g, b, samecolor){
        super(glcontext); // initialize the parent class
        
        //------------------------------ CONE ------------------------------
        //var ntriangulos = 30; // Define quantos triângulos se utilizam --> 30 triângulos em 360º
        var verticesC = [];
        var colorsC = [];
        var texCoordsC = [];    
        var numverticesC = 0;
        var angulo = 2*Math.PI/ntriangulos;
        
        var normais2 = [];
        
        for(var i = 0; i < ntriangulos; i++){
            //----------------------- TRIÂNGULO 1 - BASE -----------------------
            // Vértice "central"
            var x = 0;
            var y = 0;
            var z = 0;
            
            verticesC.push(x);
            verticesC.push(y);
            verticesC.push(z);
            
            colorsC.push(0.0);
            colorsC.push(1.0);
            colorsC.push(0.0);
            
            texCoordsC.push(0.5);
            texCoordsC.push(1);
            
            normais2.push(x);
            normais2.push(y);
            normais2.push(-1.0);
            
            
            // Vértice 1
            var x = Math.cos(angulo*i);
            var y = Math.sin(angulo*i);
            var z = 0;
            
            verticesC.push(x);
            verticesC.push(y);
            verticesC.push(z);
            
            colorsC.push(0.0);
            colorsC.push(1.0);
            colorsC.push(0.0);
            
            texCoordsC.push(i/ntriangulos);
            texCoordsC.push(0);
            
            normais2.push(x);
            normais2.push(y);
            normais2.push(-1.0);
            
            // Vértice 3
            var x = Math.cos(angulo*(i+1));
            var y = Math.sin(angulo*(i+1));
            var z = 0;
            
            verticesC.push(x);
            verticesC.push(y);
            verticesC.push(z);
            
            colorsC.push(0.0);
            colorsC.push(1.0);
            colorsC.push(0.0);
            
            texCoordsC.push((i+1)/ntriangulos);
            texCoordsC.push(0);
            
            normais2.push(x);
            normais2.push(y);
            normais2.push(-1.0);
            
            numverticesC = numverticesC + 3; // Adiciona 3 vértices
            
            //---------------------- TRIÂNGULO 1 - LATERAL ----------------------
            // Vértice 1
            var x = Math.cos(angulo*i);
            var y = Math.sin(angulo*i);
            var z = 0;

            verticesC.push(x);
            verticesC.push(y);
            verticesC.push(z);
            
            colorsC.push(1.0);
            colorsC.push(0.0);
            colorsC.push(0.0);
            
            texCoordsC.push(i/ntriangulos);
            texCoordsC.push(0);
            
            normais2.push(x);
            normais2.push(y);
            normais2.push(Math.PI / 2 - Math.atan(1/z));
            
            // Vértice 2
            var x = 0
            var y = 0
            var z = 2;
            verticesC.push(x);
            verticesC.push(y);
            verticesC.push(z);
            
            colorsC.push(1.0);
            colorsC.push(0.0);
            colorsC.push(0.0);
            
            texCoordsC.push(0.5);
            texCoordsC.push(1);
            
            normais2.push(x);
            normais2.push(y);
            normais2.push(Math.PI / 2 - Math.atan(1/z));
            
            // Vértice 3
            var x = Math.cos(angulo*(i+1));
            var y = Math.sin(angulo*(i+1));
            var z = 0;
            
            verticesC.push(x);
            verticesC.push(y);
            verticesC.push(z);
            
            colorsC.push(1.0);
            colorsC.push(0.0);
            colorsC.push(0.0);
            
            texCoordsC.push((i+1)/ntriangulos);
            texCoordsC.push(0);
            
            normais2.push(x);
            normais2.push(y);
            normais2.push(Math.PI / 2 - Math.atan(1/z));
            
            numverticesC = numverticesC + 3; // Adiciona 3 vértices
            
        }
        
        this.numvertices = numverticesC;

        if(samecolor == 1){
            var newColor = [];
            for(var i = 0; i < this.numvertices; i++){
                newColor.push(r);
                newColor.push(g);
                newColor.push(b);
            }
            colorsC = newColor;
        }
        


        
        
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        var a = new Float32Array(verticesC);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
        
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer); 
        var a = new Float32Array(colorsC);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
        
        this.texcoordsbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordsbuffer); 
        var a = new Float32Array(texCoordsC);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW); 
        
        this.normalbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.normalbuffer);    
        var a = new Float32Array(normais2);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
    
        
    }  
}


class coneL extends cone{
    
    settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
    
    drawit(viewMat, projectionMat, NoiteDia, lamp1, lamp2, lightL1, lightL2){
        this.shaderprog.startUsing();
        
        // TEXTURAS
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordsbuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 0);
        
        
        // NORMAIS
        this.normalvertex = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "VertexNormal");
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.normalbuffer);
        
        this.gl.vertexAttribPointer(this.normalvertex, // Attribute location
                           3, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           this.gl.FALSE,  // 
                           3*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the beginint of a singles vertex to this attribute

        this.gl.enableVertexAttribArray(this.normalvertex);
        
        var NormalMat = glm.mat3(this.modelMat);
        
        var MVPMatrix = this.modelMat;
        //console.log('matrix:'+MVPMatrix);                                                                       

        var ambiente = glm.vec3(0.9, 0.9, 0.9);
        var light = glm.vec3(0.0, 0.0, 0.0);
        var light1 = glm.vec3(0.0, 0.0, 0.0);
        var light2 = glm.vec3(0.0, 0.0, 0.0);                                                                     
                                                                             
        var posicao = glm.vec3(0.0, 50.0, 0.0);
        var posicao1 = glm.vec3(0.0, 4.0, -30.0);
        var posicao2 = glm.vec3(0.0, 4.0, 30.0);
                                                                             
        var shine = 1.0;                                                                     
        var streng = 100.5;
        var camera = glm.vec3(0.0, 0.0, 0.0);
        var camera1 = glm.vec3(-150.0, -150.0, -50.0);
        var camera2 = glm.vec3(-150.0, -150.0, 50.0);
        
        
        posicao1 = glm.vec3(0.0, 4.0, -30);
        posicao2 = glm.vec3(0.0, 4.0, 30.0);
        camera1 = glm.vec3(-150, -150, -50.0);
        camera2 = glm.vec3(-150, -150, 50.0);
                                                    
        // Dia
        if(NoiteDia == 1){
            ambiente = glm.vec3(0.9, 0.9, 0.9);
            light =  glm.vec3(1.0, 1.0, 0.8);
            shine = 1.0;
            streng = 100.5;
        }
        
        //Noite
        if(NoiteDia == 0 && lamp1 == 1 && lamp2 == 1){
            ambiente = glm.vec3(0.25, 0.25, 0.25);
            light = glm.vec3(0.1, 0.1, 0.1);
            light1 = lightL1;
            light2 = lightL2;
            shine = 1.0;
            streng = 10.5;

        }else if(NoiteDia == 0 && lamp1 == 1 && lamp2 == 0){
            ambiente = glm.vec3(0.25, 0.25, 0.25);
            light = glm.vec3(0.1, 0.1, 0.1);
            light1 = lightL1;
            shine = 1.0;
            streng = 10.5;

        }else if(NoiteDia == 0 && lamp1 == 0 && lamp2 == 1){
            ambiente = glm.vec3(0.25, 0.25, 0.25);
            light = glm.vec3(0.1, 0.1, 0.1);
            light2 = lightL2;
            shine = 1.0;
            streng = 10.5;

        }else if(NoiteDia == 0 && lamp1 == 0 && lamp2 == 0){
            ambiente = glm.vec3(0.25, 0.25, 0.25);
            light = glm.vec3(0.1, 0.1, 0.1); 
            shine = 1.0;
            streng = 10.5;
        }
                                                                                                                   
        var matriz_normal = this.gl.getUniformLocation(this.shaderprog.shaderProgram,  "NormalMatrix");
        this.gl.uniformMatrix3fv(matriz_normal, false, NormalMat.array);

        var mvloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "MVPMatrix");
        this.gl.uniformMatrix4fv(mvloc,false, MVPMatrix.array);                                                                     
        
        var ambiente1 = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "Ambient");
        this.gl.uniform3fv(ambiente1, ambiente.array);
        
        var light_color = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "LightColor");
        this.gl.uniform3fv(light_color, light.array);
                                                                             
        var light_color1 = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "LightColor1");
        this.gl.uniform3fv(light_color1, light1.array);                                                                     
                                                                             
        var light_color2 = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "LightColor2");
        this.gl.uniform3fv(light_color2, light2.array);                                                                     
        
        var shininess = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "Shininess");
        this.gl.uniform1f(shininess, shine);
        
        var strength = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "Strength");
        this.gl.uniform1f(strength, streng);
        
        var posic = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "LightDirection");
        this.gl.uniform3fv(posic, posicao.array);
                                                                             
        var posic1 = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "LightDirection1");
        this.gl.uniform3fv(posic1, posicao1.array);
                                                                             
        var posic2 = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "LightDirection2");
        this.gl.uniform3fv(posic2, posicao2.array);
        
        var ViewPos = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "ViewPosition");
        this.gl.uniform3fv(ViewPos, camera.array);
        
        var ViewPos1 = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "ViewPosition1");
        this.gl.uniform3fv(ViewPos1, camera1.array);
        
        var ViewPos2 = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "ViewPosition2");
        this.gl.uniform3fv(ViewPos2, camera2.array);
        
        
        // the parent method does the rest
        super.drawit(viewMat,projectionMat);
    }  
}
</script>

In [11]:
%%html
<canvas id="myCanvasCone2" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp">

class myappCone2 extends DEECapp{
    angRotacao = 0;
    
    initialize(){
          
        var fragsrc = document.getElementById("my-fragment-shaderL").text;
        var vertsrc = document.getElementById("my-vertex-shaderL").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        
        // perform other initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.67,0.923,0.99,1.0);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("Images/tronco2.jpg");
        
        // ntriangulos, r, g, b, samecolor, screen
        this.mycone = new coneL(this.gl, 30, 54/255, 31/255, 21/255, 1);
        this.mycone.settexture(this.texture);
        this.mycone.setShader(this.shaderprog);
        
    }
    
    render(){
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        
        this.angRotacao = this.angRotacao + 3;
        
        
        // TRANSLAÇÃO
        var TranslationMat = glm.translate(glm.mat4(1.0), glm.vec3(12.0,10.0,10.0));
        
        // ROTAÇÃO
        var RotMat2 = glm.rotate(glm.radians(-90), glm.vec3(1.0,0.0,0.0));
        var RotMat = glm.rotate(glm.radians(this.angRotacao), glm.vec3(1.0,0.0,0.0));

        var myModelMatrix = TranslationMat['*'](RotMat)['*'](RotMat2);
        
        // Projection and view matrices
        this.projM = glm.perspective(glm.radians(45),1,0.1,100);
        
        this.viewM = glm.lookAt(glm.vec3(5,5,10),glm.vec3(9.5,9.5,9.5),glm.vec3(0,1,0));


        this.mycone.setModelTransformation(myModelMatrix);
        
        // Utilizar o método drawit fornecido
        this.mycone.drawit(this.viewM, this.projM, 1, 0, 0);

    }
}

var appCone2 = new myappCone2('myCanvasCone2');

appCone2.run();
</script>

<h2>Cylinder:</h2>
<p>
    A loop was created, where for each iteration, four triangles are defined. First, a triangle for one base, then a triangle for the side, followed by a triangle for the other base, and another triangle for the side. The vertices for the bases are defined according to a <strong>circle</strong> given by the coordinates (x, y, z) = (cos(ang), sin(ang), 0), where <em>ang</em> is an angle that multiplies by a variable <em>i</em> that increments in the loop, allowing the traversal from 0 to <strong>2π</strong>. The base of the cylinder, that is, the circle, has a variable <strong>ntriangles</strong> that indicates the number of triangles drawn in a <strong>complete revolution</strong>. Therefore, <em>ang = 2π/ntriangles</em>. This includes <strong>textures</strong> and <strong>normals</strong> pointing outward. Here, all components (<strong>uniform variables</strong>) that will be used by the shaders are defined.
</p>

In [12]:
%%html
<script>
class cilindro extends CGRAobject{ 
    constructor(glcontext, ntriangulos, r, g, b, samecolor){
        super(glcontext); // initialize the parent class
        
        //------------------------------ CILINDRO ------------------------------
        //var ndivisoes = 100/ntriangulos;
        //var ndivisoesAlt = 0;
        
        var verticesC = [];
        var colorsC = [];
        var texCoordsC = [];
        var numverticesC = 0;
        var angulo = 2*Math.PI/ntriangulos;
        
        var normais2 = [];
        
        for(var i = 0; i < ntriangulos; i++){
            //----------------------- TRIÂNGULO 1 - BASE -----------------------
            // Vértice "central"
            var x = 0;
            var y = 0;
            var z = 0;
            
            verticesC.push(x);
            verticesC.push(y);
            verticesC.push(z);
            
            colorsC.push(0.0);
            colorsC.push(1.0);
            colorsC.push(0.0);
            
            texCoordsC.push(0.5);
            texCoordsC.push(1);
            
            normais2.push(x);
            normais2.push(y);
            normais2.push(-1.0);
            
            // Vértice 1
            var x = Math.cos(angulo*i);
            var y = Math.sin(angulo*i);
            var z = 0;
            
            verticesC.push(x);
            verticesC.push(y);
            verticesC.push(z);
            
            colorsC.push(0.0);
            colorsC.push(1.0);
            colorsC.push(0.0);
            
            texCoordsC.push(i/ntriangulos);
            texCoordsC.push(0.0);
            
            normais2.push(x);
            normais2.push(y);
            normais2.push(-1.0);
            
            // Vértice 3
            var x = Math.cos(angulo*(i+1));
            var y = Math.sin(angulo*(i+1));
            var z = 0;
            
            verticesC.push(x);
            verticesC.push(y);
            verticesC.push(z);
            
            colorsC.push(0.0);
            colorsC.push(1.0);
            colorsC.push(0.0);
            
            texCoordsC.push((i+1)/ntriangulos);
            texCoordsC.push(0.0);
            
            normais2.push(x);
            normais2.push(y);
            normais2.push(-1.0);
            
            numverticesC = numverticesC + 3; // Adiciona 3 vértices
            
            //---------------------- TRIÂNGULO 1 - LATERAL ----------------------
            // Vértice 1
            var x = Math.cos(angulo*i);
            var y = Math.sin(angulo*i);
            var z = 0;

            verticesC.push(x);
            verticesC.push(y);
            verticesC.push(z);
            
            colorsC.push(1.0);
            colorsC.push(0.0);
            colorsC.push(0.0);
            
            texCoordsC.push(i/ntriangulos);
            texCoordsC.push(0);
            
            normais2.push(x);
            normais2.push(y);
            normais2.push(Math.PI / 2 - Math.atan(1/z));
            
            // Vértice 2
            var z = 2;
            verticesC.push(x);
            verticesC.push(y);
            verticesC.push(z);
            
            colorsC.push(1.0);
            colorsC.push(0.0);
            colorsC.push(0.0);
            
            texCoordsC.push(i/ntriangulos);
            texCoordsC.push(1);
            
            normais2.push(x);
            normais2.push(y);
            normais2.push(Math.PI / 2 - Math.atan(1/z));
            
            // Vértice 3
            var x = Math.cos(angulo*(i+1));
            var y = Math.sin(angulo*(i+1));
            var z = 0;
            
            verticesC.push(x);
            verticesC.push(y);
            verticesC.push(z);
            
            colorsC.push(1.0);
            colorsC.push(0.0);
            colorsC.push(0.0);
            
            texCoordsC.push((i+1)/ntriangulos);
            texCoordsC.push(0);
            
            normais2.push(x);
            normais2.push(y);
            normais2.push(Math.PI / 2 - Math.atan(1/z));
            
            numverticesC = numverticesC + 3; // Adiciona 3 vértices
            
            //----------------------- TRIÂNGULO 2 - BASE -----------------------
            // Vértice "central"
            var x = 0;
            var y = 0;
            var z = 2;
            
            verticesC.push(x);
            verticesC.push(y);
            verticesC.push(z);
            
            colorsC.push(0.0);
            colorsC.push(1.0);
            colorsC.push(1.0);
            
            texCoordsC.push(0.5);
            texCoordsC.push(1);  
            
            normais2.push(x);
            normais2.push(y);
            normais2.push(1.0);
            
            // Vértice 2
            var x = Math.cos(angulo*i);
            var y = Math.sin(angulo*i);
            var z = 2;

            verticesC.push(x);
            verticesC.push(y);
            verticesC.push(z);
            
            colorsC.push(0.0);
            colorsC.push(1.0);
            colorsC.push(1.0);
            
            texCoordsC.push(i/ntriangulos);
            texCoordsC.push(0.0);
            
            normais2.push(x);
            normais2.push(y);
            normais2.push(1.0);
            
            // Vértice 4
            var x = Math.cos(angulo*(i+1));
            var y = Math.sin(angulo*(i+1));
            var z = 2;
            verticesC.push(x);
            verticesC.push(y);
            verticesC.push(z);
            
            colorsC.push(0.0);
            colorsC.push(1.0);
            colorsC.push(1.0);
            
            texCoordsC.push((i+1)/ntriangulos);
            texCoordsC.push(0.0);
            
            normais2.push(x);
            normais2.push(y);
            normais2.push(1.0);
            
            numverticesC = numverticesC + 3; // Adiciona 3 vértices
            
            //---------------------- TRIÂNGULO 2 - LATERAL ----------------------
            // Vértice 2
            var x = Math.cos(angulo*i);
            var y = Math.sin(angulo*i);
                    
            var z = 2;

            verticesC.push(x);
            verticesC.push(y);
            verticesC.push(z);
            
            colorsC.push(0.0);
            colorsC.push(0.0);
            colorsC.push(1.0);
            
            texCoordsC.push(i/ntriangulos);
            texCoordsC.push(1);
            
            normais2.push(x);
            normais2.push(y);
            normais2.push(Math.PI / 2 - Math.atan(1/z));
            
            // Vértice 3
            var x = Math.cos(angulo*(i+1));
            var y = Math.sin(angulo*(i+1));
            var z = 0;
            verticesC.push(x);
            verticesC.push(y);
            verticesC.push(z);
            
            colorsC.push(0.0);
            colorsC.push(0.0);
            colorsC.push(1.0);
            
            texCoordsC.push((i+1)/ntriangulos);
            texCoordsC.push(0);
            
            normais2.push(x);
            normais2.push(y);
            normais2.push(Math.PI / 2 - Math.atan(1/z));
            
            // Vértice 4
            var z = 2;
            
            verticesC.push(x);
            verticesC.push(y);
            verticesC.push(z);
            
            colorsC.push(0.0);
            colorsC.push(0.0);
            colorsC.push(1.0);
            
            texCoordsC.push((i+1)/ntriangulos);
            texCoordsC.push(1);
            
            normais2.push(x);
            normais2.push(y);
            normais2.push(Math.PI / 2 - Math.atan(1/z));
            
            numverticesC = numverticesC + 3; // Adiciona 3 vértices
            
        }
        
        this.numvertices = numverticesC;
        
        if(samecolor == 1){
            var newColor = [];
            for(var i = 0; i < this.numvertices; i++){
                newColor.push(r);
                newColor.push(g);
                newColor.push(b);
            }
            colorsC = newColor;
        }
        
        var normais = [];

        for (var i = 0; i < this.numvertices; i++){
            x = i * 3;
            normais.push(verticesC[x]);
            normais.push(verticesC[x+1]);
            normais.push(0);
        }
        
        
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        var a = new Float32Array(verticesC);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
        
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer); 
        var a = new Float32Array(colorsC);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
        
        this.texcoordsbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordsbuffer); 
        var a = new Float32Array(texCoordsC);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW); 
        
        this.normalbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.normalbuffer);    
        var a = new Float32Array(normais2);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
    
        
    }  
}


class cilindroL extends cilindro{
    
    settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
    
    drawit(viewMat, projectionMat, NoiteDia, lamp1, lamp2, lightL1, lightL2){
        this.shaderprog.startUsing();
        
        // TEXTURAS
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordsbuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 0);
        
        
        // NORMAIS
        this.normalvertex = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "VertexNormal");
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.normalbuffer);
        
        this.gl.vertexAttribPointer(this.normalvertex, // Attribute location
                           3, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           this.gl.FALSE,  // 
                           3*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the beginint of a singles vertex to this attribute

        this.gl.enableVertexAttribArray(this.normalvertex);
        
        var NormalMat = glm.mat3(this.modelMat);
        
        var MVPMatrix = this.modelMat;
        //console.log('matrix:'+MVPMatrix);                                                                       

        var ambiente = glm.vec3(0.9, 0.9, 0.9);
        var light = glm.vec3(0.0, 0.0, 0.0);
        var light1 = glm.vec3(0.0, 0.0, 0.0);
        var light2 = glm.vec3(0.0, 0.0, 0.0);                                                                     
                                                                             
        var posicao = glm.vec3(0.0, 50.0, 0.0);
        var posicao1 = glm.vec3(0.0, 4.0, -30.0);
        var posicao2 = glm.vec3(0.0, 4.0, 30.0);
                                                                             
        var shine = 1.0;                                                                     
        var streng = 100.5;
        var camera = glm.vec3(0.0, 0.0, 0.0);
        var camera1 = glm.vec3(-150.0, -150.0, -50.0);
        var camera2 = glm.vec3(-150.0, -150.0, 50.0);
        
        
        posicao1 = glm.vec3(0.0, 4.0, -30);
        posicao2 = glm.vec3(0.0, 4.0, 30.0);
        camera1 = glm.vec3(-150, -150, -50.0);
        camera2 = glm.vec3(-150, -150, 50.0);
                                                    
        // Dia
        if(NoiteDia == 1){
            ambiente = glm.vec3(0.9, 0.9, 0.9);
            light =  glm.vec3(1.0, 1.0, 0.8);
            shine = 1.0;
            streng = 100.5;
        }
        
        //Noite
        if(NoiteDia == 0 && lamp1 == 1 && lamp2 == 1){
            ambiente = glm.vec3(0.25, 0.25, 0.25);
            light = glm.vec3(0.1, 0.1, 0.1);
            light1 = lightL1;
            light2 = lightL2;
            shine = 1.0;
            streng = 10.5;

        }else if(NoiteDia == 0 && lamp1 == 1 && lamp2 == 0){
            ambiente = glm.vec3(0.25, 0.25, 0.25);
            light = glm.vec3(0.1, 0.1, 0.1);
            light1 = lightL1;
            shine = 1.0;
            streng = 10.5;

        }else if(NoiteDia == 0 && lamp1 == 0 && lamp2 == 1){
            ambiente = glm.vec3(0.25, 0.25, 0.25);
            light = glm.vec3(0.1, 0.1, 0.1);
            light2 = lightL2;
            shine = 1.0;
            streng = 10.5;

        }else if(NoiteDia == 0 && lamp1 == 0 && lamp2 == 0){
            ambiente = glm.vec3(0.25, 0.25, 0.25);
            light = glm.vec3(0.1, 0.1, 0.1); 
            shine = 1.0;
            streng = 10.5;
        }
                                                                                                                   
        var matriz_normal = this.gl.getUniformLocation(this.shaderprog.shaderProgram,  "NormalMatrix");
        this.gl.uniformMatrix3fv(matriz_normal, false, NormalMat.array);

        var mvloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "MVPMatrix");
        this.gl.uniformMatrix4fv(mvloc,false, MVPMatrix.array);                                                                     
        
        var ambiente1 = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "Ambient");
        this.gl.uniform3fv(ambiente1, ambiente.array);
        
        var light_color = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "LightColor");
        this.gl.uniform3fv(light_color, light.array);
                                                                             
        var light_color1 = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "LightColor1");
        this.gl.uniform3fv(light_color1, light1.array);                                                                     
                                                                             
        var light_color2 = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "LightColor2");
        this.gl.uniform3fv(light_color2, light2.array);                                                                     
        
        var shininess = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "Shininess");
        this.gl.uniform1f(shininess, shine);
        
        var strength = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "Strength");
        this.gl.uniform1f(strength, streng);
        
        var posic = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "LightDirection");
        this.gl.uniform3fv(posic, posicao.array);
                                                                             
        var posic1 = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "LightDirection1");
        this.gl.uniform3fv(posic1, posicao1.array);
                                                                             
        var posic2 = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "LightDirection2");
        this.gl.uniform3fv(posic2, posicao2.array);
        
        var ViewPos = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "ViewPosition");
        this.gl.uniform3fv(ViewPos, camera.array);
        
        var ViewPos1 = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "ViewPosition1");
        this.gl.uniform3fv(ViewPos1, camera1.array);
        
        var ViewPos2 = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "ViewPosition2");
        this.gl.uniform3fv(ViewPos2, camera2.array);
        
        

        
        
        // the parent method does the rest
        super.drawit(viewMat,projectionMat);
    }  
}
</script>

In [13]:
%%html
<canvas id="myCanvasCilindro2" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp">

class myappCilindro2 extends DEECapp{
    angRotacao = 0;
    
    initialize(){
          
        var fragsrc = document.getElementById("my-fragment-shaderL").text;
        var vertsrc = document.getElementById("my-vertex-shaderL").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        
        // perform other initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.67,0.923,0.99,1.0);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("Images/tronco2.jpg");
        
        // ntriangulos, r, g, b, samecolor, screen
        this.mycilindro = new cilindroL(this.gl, 30, 54/255, 31/255, 21/255, 1);
        this.mycilindro.settexture(this.texture);
        this.mycilindro.setShader(this.shaderprog);
        
    }
    
    render(){
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        
        this.angRotacao = this.angRotacao + 3;
        
        
        // TRANSLAÇÃO
        var TranslationMat = glm.translate(glm.mat4(1.0), glm.vec3(12.0,10.0,10.0));
        
        // ROTAÇÃO
        var RotMat2 = glm.rotate(glm.radians(90), glm.vec3(1.0,0.0,0.0));
        var RotMat = glm.rotate(glm.radians(this.angRotacao), glm.vec3(0.0,0.0,1.0));

        var myModelMatrix = TranslationMat['*'](RotMat)['*'](RotMat2);
        
        // Projection and view matrices
        this.projM = glm.perspective(glm.radians(45),1,0.1,100);
        
        this.viewM = glm.lookAt(glm.vec3(4,4,10),glm.vec3(9.5,9.5,9.5),glm.vec3(0,1,0));


        this.mycilindro.setModelTransformation(myModelMatrix);
        
        // Utilizar o método drawit fornecido
        this.mycilindro.drawit(this.viewM, this.projM, 1, 0, 0);

    }
}

var appCilindro2 = new myappCilindro2('myCanvasCilindro2');

appCilindro2.run();
</script>

<h2>Sphere:</h2>
<p>
    For the sphere, spherical coordinates are used that vary as follows:
    <ul>
        <li><strong>phi</strong> - 0 to 2π</li>
        <li><strong>theta</strong> - 0 to π</li>
        <li><strong>radius</strong> = 2</li>
    </ul>
    These coordinates are converted to Cartesian coordinates using the formulas:
    <ul>
        <li><em>x = r * sin(theta) * cos(phi)</em></li>
        <li><em>y = r * sin(theta) * sin(phi)</em></li>
        <li><em>z = r * cos(theta)</em></li>
    </ul>
    Thus, there are two loops, in which one varies <strong>phi</strong> from 0 to 2π and <strong>theta</strong> from 0 to π. In each iteration of this loop, two triangles are drawn between two vertices of the current circle (with <em>theta * i; phi * j; phi * j+1</em>) and two vertices of the next circle (with <em>theta * i+1; phi * j; phi * j+1</em>).
</p>
<p>
    Again, we have the variable <strong>ntriangles</strong> that indicates the number of triangles drawn in a <strong>co


In [14]:
%%html
<script>
class esfera extends CGRAobject{ 
    constructor(glcontext, ntriangulos, r, g, b, samecolor){
        super(glcontext); // initialize the parent class
        
         //------------------------------ ESFERA ------------------------------
        var vertices = [];
        var colors = [];
        var texCoords = [];
        this.numvertices = 0;
            
        //var ntriangulos = 30; // Define quantos triângulos se utilizam --> 30 triângulos em 360º
        
        // Coordenadas esféricas: P(r, theta, phi);
        var raio = 2;
        var theta = Math.PI/ntriangulos;
        var phi = 2*Math.PI/ntriangulos;


        // De coordenadas esféricas para cartesinas
        // x = r * sin(theta) * cos(phi)
        // y = r * sin(theta) * sin(phi)
        // z = r * cos(theta)
        
        // Guarda as coordenadas de 30 círculos "rodados", cada círculo com 30 pontos
        // Também guarda as cores em vários tons de verde para se utilizarem em árvores
        // e para uma melhor visualização
        for (var i = 0; i < ntriangulos; i++){
            for (var j = 0; j < ntriangulos; j++){
                
                //-------------------- VÉRTICES CÍRCULO 1 --------------------
                var x1 = raio * Math.sin(theta*i) * Math.cos(phi*j);
                var y1 = raio * Math.sin(theta*i) * Math.sin(phi*j);
                var z1 = raio * Math.cos(theta*i);
                
                var x12 = raio * Math.sin(theta*i) * Math.cos(phi*(j+1));
                var y12 = raio * Math.sin(theta*i) * Math.sin(phi*(j+1));
                
                //-------------------- VÉRTICES CÍRCULO 2 --------------------
                var x2 = raio * Math.sin(theta*(i+1)) * Math.cos(phi*j);
                var y2 = raio * Math.sin(theta*(i+1)) * Math.sin(phi*j);
                var z2 = raio * Math.cos(theta*(i+1));
                
                var x22 = raio * Math.sin(theta*(i+1)) * Math.cos(phi*(j+1));
                var y22 = raio * Math.sin(theta*(i+1)) * Math.sin(phi*(j+1));
                
                //------------------------ TRIÂNGULO 1 ------------------------
                
                // Círculo 1 - Vértice 1 
                vertices.push(x1);
                vertices.push(y1);
                vertices.push(z1);
                
                colors.push(0);
                colors.push(0.4);
                colors.push(0);
                
                texCoords.push(i/ntriangulos);
                texCoords.push(j/ntriangulos);
                
                // Círculo 1 - Vértice 2
                vertices.push(x12);
                vertices.push(y12);
                vertices.push(z1);
                
                colors.push(0);
                colors.push(0.5);
                colors.push(0);
                
                texCoords.push(i/ntriangulos);
                texCoords.push((j+1)/ntriangulos);
                
                // Círculo 2 - Vértice 1
                vertices.push(x2);
                vertices.push(y2);
                vertices.push(z2);
                
                colors.push(0);
                colors.push(0.5);
                colors.push(0);
                
                texCoords.push((i+1)/ntriangulos);
                texCoords.push(j/ntriangulos);
                
                //------------------------ TRIÂNGULO 2 ------------------------
                // Círculo 1 - Vértice 2 
                vertices.push(x12);
                vertices.push(y12);
                vertices.push(z1);
                
                colors.push(0);
                colors.push(0.5);
                colors.push(0.1);
                
                texCoords.push(i/ntriangulos);
                texCoords.push((j+1)/ntriangulos);
                
                // Círculo 2 - Vértice 1 
                vertices.push(x2);
                vertices.push(y2);
                vertices.push(z2);
                
                colors.push(0);
                colors.push(0.4);
                colors.push(0);
                
                texCoords.push((i+1)/ntriangulos);
                texCoords.push(j/ntriangulos);
                
                // Círculo 2 - Vértice 2 
                vertices.push(x22);
                vertices.push(y22);
                vertices.push(z2);
                
                colors.push(0);
                colors.push(0.5);
                colors.push(0);
                
                texCoords.push((i+1)/ntriangulos);
                texCoords.push((j+1)/ntriangulos);
                
                this.numvertices = this.numvertices + 6;
                
            }
        } 
        
        if(samecolor == 1){
            var newColor = [];
            for(var i = 0; i < this.numvertices; i++){
                newColor.push(r);
                newColor.push(g);
                newColor.push(b);
            }
            colors = newColor;
        }
        
        var normais = [];

        for (var i = 0; i <= this.numvertices * 3; i++){
            normais.push(vertices[i]);
        }
        
        
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        var a = new Float32Array(vertices);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
        
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer); 
        var a = new Float32Array(colors);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
        
        this.texcoordsbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordsbuffer); 
        var a = new Float32Array(texCoords);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW); 
        
        this.normalbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.normalbuffer);    
        var a = new Float32Array(normais);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
    
        
    }  
}


class esferaL extends esfera{
    
    settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
    
    drawit(viewMat, projectionMat, NoiteDia, lamp1, lamp2, lightL1, lightL2){ // myModelMatrix
        this.shaderprog.startUsing();
        
        // TEXTURAS
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordsbuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 0);
        
        
        // NORMAIS
        this.normalvertex = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "VertexNormal");
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.normalbuffer);
        
        this.gl.vertexAttribPointer(this.normalvertex, // Attribute location
                           3, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           this.gl.FALSE,  // 
                           3*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the beginint of a singles vertex to this attribute

        this.gl.enableVertexAttribArray(this.normalvertex);
                                                                             
      var NormalMat = glm.mat3(this.modelMat);
        
        var MVPMatrix = this.modelMat;
        //console.log('matrix:'+MVPMatrix);                                                                       

        var ambiente = glm.vec3(0.9, 0.9, 0.9);
        var light = glm.vec3(0.0, 0.0, 0.0);
        var light1 = glm.vec3(0.0, 0.0, 0.0);
        var light2 = glm.vec3(0.0, 0.0, 0.0);                                                                     
                                                                             
        var posicao = glm.vec3(0.0, 50.0, 0.0);
        var posicao1 = glm.vec3(0.0, 4.0, -30.0);
        var posicao2 = glm.vec3(0.0, 4.0, 30.0);
                                                                             
        var shine = 1.0;                                                                     
        var streng = 100.5;
        var camera = glm.vec3(0.0, 0.0, 0.0);
        var camera1 = glm.vec3(-150.0, -150.0, -50.0);
        var camera2 = glm.vec3(-150.0, -150.0, 50.0);
        
        
        posicao1 = glm.vec3(0.0, 4.0, -30);
        posicao2 = glm.vec3(0.0, 4.0, 30.0);
        camera1 = glm.vec3(-150, -150, -50.0);
        camera2 = glm.vec3(-150, -150, 50.0);
                                                    
        // Dia
        if(NoiteDia == 1){
            ambiente = glm.vec3(0.9, 0.9, 0.9);
            light =  glm.vec3(1.0, 1.0, 0.8);
            shine = 1.0;
            streng = 100.5;
        }
        
        //Noite
        if(NoiteDia == 0 && lamp1 == 1 && lamp2 == 1){
            ambiente = glm.vec3(0.25, 0.25, 0.25);
            light = glm.vec3(0.1, 0.1, 0.1);
            light1 = lightL1;
            light2 = lightL2;
            shine = 1.0;
            streng = 10.5;

        }else if(NoiteDia == 0 && lamp1 == 1 && lamp2 == 0){
            ambiente = glm.vec3(0.25, 0.25, 0.25);
            light = glm.vec3(0.1, 0.1, 0.1);
            light1 = lightL1;
            shine = 1.0;
            streng = 10.5;

        }else if(NoiteDia == 0 && lamp1 == 0 && lamp2 == 1){
            ambiente = glm.vec3(0.25, 0.25, 0.25);
            light = glm.vec3(0.1, 0.1, 0.1);
            light2 = lightL2;
            shine = 1.0;
            streng = 10.5;

        }else if(NoiteDia == 0 && lamp1 == 0 && lamp2 == 0){
            ambiente = glm.vec3(0.25, 0.25, 0.25);
            light = glm.vec3(0.1, 0.1, 0.1); 
            shine = 1.0;
            streng = 10.5;
        }
                                                                                                                   
        var matriz_normal = this.gl.getUniformLocation(this.shaderprog.shaderProgram,  "NormalMatrix");
        this.gl.uniformMatrix3fv(matriz_normal, false, NormalMat.array);

        var mvloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "MVPMatrix");
        this.gl.uniformMatrix4fv(mvloc,false, MVPMatrix.array);                                                                     
        
        var ambiente1 = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "Ambient");
        this.gl.uniform3fv(ambiente1, ambiente.array);
        
        var light_color = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "LightColor");
        this.gl.uniform3fv(light_color, light.array);
                                                                             
        var light_color1 = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "LightColor1");
        this.gl.uniform3fv(light_color1, light1.array);                                                                     
                                                                             
        var light_color2 = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "LightColor2");
        this.gl.uniform3fv(light_color2, light2.array);                                                                     
        
        var shininess = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "Shininess");
        this.gl.uniform1f(shininess, shine);
        
        var strength = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "Strength");
        this.gl.uniform1f(strength, streng);
        
        var posic = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "LightDirection");
        this.gl.uniform3fv(posic, posicao.array);
                                                                             
        var posic1 = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "LightDirection1");
        this.gl.uniform3fv(posic1, posicao1.array);
                                                                             
        var posic2 = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "LightDirection2");
        this.gl.uniform3fv(posic2, posicao2.array);
        
        var ViewPos = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "ViewPosition");
        this.gl.uniform3fv(ViewPos, camera.array);
        
        var ViewPos1 = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "ViewPosition1");
        this.gl.uniform3fv(ViewPos1, camera1.array);
        
        var ViewPos2 = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "ViewPosition2");
        this.gl.uniform3fv(ViewPos2, camera2.array);
        
        
        // the parent method does the rest
        super.drawit(viewMat,projectionMat);
    }  
}
</script>

In [15]:
%%html
<canvas id="myCanvasEsfera2" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp">

class myappEsfera2 extends DEECapp{
    angRotacao = 0;
    
    initialize(){
          
        var fragsrc = document.getElementById("my-fragment-shaderL").text;
        var vertsrc = document.getElementById("my-vertex-shaderL").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        
        // perform other initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.67,0.923,0.99,1.0);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("Images/folhas.jpg");
        
        // ntriangulos, r, g, b, samecolor, screen
        this.myesfera = new esferaL(this.gl, 30, 21/255,62/255,42/255, 1);
        this.myesfera.settexture(this.texture);
        this.myesfera.setShader(this.shaderprog);
        
    }
    
    render(){
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        
        this.angRotacao = this.angRotacao + 3;
        
        
        // TRANSLAÇÃO
        var TranslationMat = glm.translate(glm.mat4(1.0), glm.vec3(12.0,10.0,10.0));
        
        // ROTAÇÃO
        var RotMat2 = glm.rotate(glm.radians(90), glm.vec3(1.0,0.0,0.0));
        var RotMat = glm.rotate(glm.radians(this.angRotacao), glm.vec3(0.0,1.0,0.0));

        var myModelMatrix = TranslationMat['*'](RotMat)['*'](RotMat2);
        
        // Projection and view matrices
        this.projM = glm.perspective(glm.radians(45),1,0.1,100);
        
        this.viewM = glm.lookAt(glm.vec3(4,4,10),glm.vec3(9.5,9.5,9.5),glm.vec3(0,1,0));


        this.myesfera.setModelTransformation(myModelMatrix);
        
        // Utilizar o método drawit fornecido
        this.myesfera.drawit(this.viewM, this.projM, 1, 0, 0);

    }
}

var appEsfera2 = new myappEsfera2('myCanvasEsfera2');

appEsfera2.run();
</script>

<h1>Revolution Solids</h1>

<p>
    For this exercise, 4 vertices are initially defined. These original vertices are copied to the variables <strong>newV1</strong>, <strong>newV2</strong>, <strong>newV3</strong>, and <strong>newV4</strong>. The current vertices and the next (rotated) vertices are stored in the variable <strong>revolutionVertices</strong>.
</p>

<p>
    The next vertices are calculated through a transformation, which consists of a translation to the origin + rotation + translation from the origin. The rotation is around the Z axis, so when translating from the origin to the next position, it will be done according to the angle <strong>angRotacao</strong>.
</p>

<p>
    Now that we have a copy of the current vertices and the next vertices in <strong>revolutionVertices</strong>, a loop is performed to rearrange these vertices in the variable <strong>this.vertices</strong> to correctly connect the triangles between the vertices. That is:
</p>

<ol>
    <li>1, 2, 5</li>
    <li>2, 5, 6</li>
    <li>2, 3, 6...</li>
</ol>

<p>
    It is important to note that the "copy" of every 4 vertices occurs every 0.5 seconds.
</p>


In [16]:
%%html

<pre id="matrix"></pre>

<script>

class rotSolid extends CGRAobject{
    
    angRotacao = 15; // Ângulo de rotação
    positionX = 2; 
    positionY = 0; 
    positionZ = 0;
    rotate = 0;
    
    counter = 0;
    
    nciclos = 0;
    
    segundos = Date.now()/1000;
    segundosAfter = 0;
    
    numRevolution = 0;
    revolutionVertices = [];
    vertices = [];
    colors = [];
    
    newV1 = glm.vec4(0, 0, 0, 0);
    newV2 = glm.vec4(0, 0, 0, 0);
    newV3 = glm.vec4(0, 0, 0, 0);
    newV4 = glm.vec4(0, 0, 0, 0);
    
    constructor(glcontext){
        super(glcontext); // initialize the parent class
        
        // Definição dos vértices "originais"
        this.newV1 = glm.vec4(0.7, 0.0, 1.0, 1);
        this.newV2 = glm.vec4(0.3, 0.0, 2.0, 1);
        this.newV3 = glm.vec4(-0.3, 0.0, 3.0, 1);
        this.newV4 = glm.vec4(0.4, 0.0, 4.0, 1);
        
        
        while(1){
            
            this.segundosAfter = Date.now()/1000;
            
            
            //if(this.segundos + 0.5 <= this.segundosAfter && this.counter < 25){ 
            if(this.counter < 25){
                                                                                
                this.counter++;
                this.segundos = this.segundosAfter;
            
                //----------------------- TRIÂNGULO AZUL -----------------------

                // MATRIZ IDENTIDADE onde se aplicará todas as operações
                var matIdentidade = glm.mat4(1.0);

                //------------- VETORES DA ROTAÇÃO 1 -------------
                this.revolutionVertices[0] = (this.newV1[0]);
                this.revolutionVertices[1] = (this.newV1[1]);
                this.revolutionVertices[2] = (this.newV1[2]);

                this.revolutionVertices[3] = (this.newV2[0]);
                this.revolutionVertices[4] = (this.newV2[1]);
                this.revolutionVertices[5] = (this.newV2[2]);

                this.revolutionVertices[6] = (this.newV3[0]);
                this.revolutionVertices[7] = (this.newV3[1]);
                this.revolutionVertices[8] = (this.newV3[2]);

                this.revolutionVertices[9] = (this.newV4[0]);
                this.revolutionVertices[10] = (this.newV4[1]);
                this.revolutionVertices[11] = (this.newV4[2]);

                // Aumenta-se aqui o ângulo
                //this.angRotacao += 15;

                //------------- VETORES DA ROTAÇÃO 2 -------------
                // TRANSLAÇÃO para a origem
                var matTranslation1 = glm.mat4(1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, -this.positionX, -this.positionY, 0, 1);
                matTranslation1 = matTranslation1['*'](matIdentidade);

                // ROTAÇÃO em torno da origem
                var rotacao = glm.angleAxis(glm.radians(this.angRotacao),glm.vec3(0, 0, 1)); //Define qual o eixo de rotação: eixo dos Z e a cada instante mudo o valor do ângulo (em graus)
                var matRotacao = glm.toMat4(rotacao);
                matRotacao = matRotacao['*'](matTranslation1);

                this.positionX = 2*Math.cos(glm.radians((this.angRotacao)));
                this.positionY = 2*Math.sin(glm.radians((this.angRotacao)));

                // SEGUNDA TRANSLAÇÃO 
                var matTranslation2 = glm.mat4(1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, this.positionX, this.positionY, 0, 1);
                matTranslation2 = matTranslation2['*'](matRotacao);   


                this.newV1 = matTranslation2['*'](this.newV1);
                this.newV2 = matTranslation2['*'](this.newV2);
                this.newV3 = matTranslation2['*'](this.newV3);
                this.newV4 = matTranslation2['*'](this.newV4);

                this.revolutionVertices[12] = (this.newV1[0]);
                this.revolutionVertices[13] = (this.newV1[1]);
                this.revolutionVertices[14] = (this.newV1[2]);

                this.revolutionVertices[15] = (this.newV2[0]);
                this.revolutionVertices[16] = (this.newV2[1]);
                this.revolutionVertices[17] = (this.newV2[2]);

                this.revolutionVertices[18] = (this.newV3[0]);
                this.revolutionVertices[19] = (this.newV3[1]);
                this.revolutionVertices[20] = (this.newV3[2]);

                this.revolutionVertices[21] = (this.newV4[0]);
                this.revolutionVertices[22] = (this.newV4[1]);
                this.revolutionVertices[23] = (this.newV4[2]);


                // Para unir 4 vértices de um ângulo para os 4 vértices do ângulo seguinte,
                // é necessário desenhar 6 triângulos, ou seja, 18 vértices, cada um com 3
                // componentes: X, Y e Z


                for(var i = 0; i < 3; i++){
                    // Triângulo 1
                    this.vertices[this.nciclos*18] = this.revolutionVertices[i*3];
                    this.vertices[this.nciclos*18+1] = this.revolutionVertices[i*3+1];
                    this.vertices[this.nciclos*18+2] = this.revolutionVertices[i*3+2];

                    this.vertices[this.nciclos*18+3] = this.revolutionVertices[i*3+3];
                    this.vertices[this.nciclos*18+4] = this.revolutionVertices[i*3+4];
                    this.vertices[this.nciclos*18+5] = this.revolutionVertices[i*3+5];

                    this.vertices[this.nciclos*18+6] = this.revolutionVertices[i*3+12];
                    this.vertices[this.nciclos*18+7] = this.revolutionVertices[i*3+13];
                    this.vertices[this.nciclos*18+8] = this.revolutionVertices[i*3+14];

                     // Triângulo 2
                    this.vertices[this.nciclos*18+9] = this.revolutionVertices[i*3+3];
                    this.vertices[this.nciclos*18+10] = this.revolutionVertices[i*3+4];
                    this.vertices[this.nciclos*18+11] = this.revolutionVertices[i*3+5];

                    this.vertices[this.nciclos*18+12] = this.revolutionVertices[i*3+12];
                    this.vertices[this.nciclos*18+13] = this.revolutionVertices[i*3+13];
                    this.vertices[this.nciclos*18+14] = this.revolutionVertices[i*3+14];

                    this.vertices[this.nciclos*18+15] = this.revolutionVertices[i*3+15];
                    this.vertices[this.nciclos*18+16] = this.revolutionVertices[i*3+16];
                    this.vertices[this.nciclos*18+17] = this.revolutionVertices[i*3+17];

                    this.numRevolution += 6;

                    this.nciclos ++;

                }

                for(var i = 0; i < this.numRevolution*3; i = i + 3){
                    
                    if(i%2 == 0){
                        this.colors[i] = 0;
                        this.colors[i+1] = 0;
                        this.colors[i+2] = 1;

                    }else{
                        this.colors[i] = 1;
                        this.colors[i+1] = 0;
                        this.colors[i+2] = 0;
                    }

                }

                this.numvertices =  this.numRevolution;
            }
            
            if(this.counter >= 25){
                break;
            }
        }
        
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.vertices), this.gl.STATIC_DRAW);

        
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.colors), this.gl.STATIC_DRAW);
        
        
    }
}


</script>

<canvas id="exercise_rot" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>

<script id="myapp_rot">

class myapp_rot extends DEECapp{
    
    rotate = 0;
    
    initialize(){
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        
        this.gl.enable(this.gl.DEPTH_TEST);
        
        // Create object
        this.solid = new rotSolid(this.gl);
        this.solid.setShader(this.shaderprog);

    }
    
    render(){
       
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        
        
        //------------------------- SÓLIDO A RODAR -------------------------
        
        this.rotate += 5;
        
        // TRANSLAÇÃO
        var TranslationMat = glm.translate(glm.mat4(1.0), glm.vec3(13.0,17.0,18.0));
        
        // ROTAÇÃO
        var RotMat = glm.rotate(glm.degrees(60.0), glm.vec3(0.0,1.0,0.0));
        var RotMat2 = glm.rotate(glm.radians(this.rotate), glm.vec3(0.0,0.0,1.0));
        
        var myModelMatrix = TranslationMat['*'](RotMat['*'](RotMat2));

        // TRANSFORMAÇÃO DE VISTA/PERSPETIVA
        var ViewMatrix = glm.lookAt(glm.vec3(4,3,3), glm.vec3(9.5,9.5,9.5), glm.vec3(0,1,0));
        
        // PROJEÇÃO - FoV, aspectratio, nearplane, farplane
        var ProjectionMatrix = glm.perspective(glm.radians(45.0), 1.0, 0.1, 100.0);
        
        this.solid.setModelTransformation(myModelMatrix);
        
        // Utilizar o método drawit fornecido
        this.solid.drawit(ViewMatrix, ProjectionMatrix);

    }
}

var app_rot = new myapp_rot('exercise_rot');

app_rot.run();
</script>

<h1>Extrusion Shapes</h1>

<p>
    This exercise used the same logic as the previous one, with the difference that only 3 initial vertices were used and the rearrangement of the vertices is different for the extrusion effect. Additionally, a rotation around the Z axis is performed, where the value of Z increases, giving the effect of a "spring" to the created shape.
</p>

<p>
    The vertices are connected in the following order:
</p>

<ol>
    <li>1, 2, 4</li>
    <li>2, 4, 5</li>
    <li>2, 3, 5</li>
    <li>3, 5, 6</li>
    <li>3, 1, 4</li>
    <li>1, 4, 6</li>
</ol>

In [17]:
%%html

<script>

class extSolid extends CGRAobject{
    
    angRotacao = 15; // Ângulo de rotação
    positionX = 2; 
    positionY = 0; 
    positionZ = 0;
    rotate = 0;
    
    counter = 0;
    
    nciclos = 0;
    
    numRevolution = 0;
    revolutionVertices = [];
    vertices = [];
    colors = [];
    
    newV1 = glm.vec4(0, 0, 0, 0);
    newV2 = glm.vec4(0, 0, 0, 0);
    newV3 = glm.vec4(0, 0, 0, 0);
    
    constructor(glcontext){
        super(glcontext); // initialize the parent class
        
        // Definição dos vértices "originais"
        this.newV1 = glm.vec4(0.0, 0, 1.0, 1);
        this.newV2 = glm.vec4(0.0, 0.5, 2.0, 1);
        this.newV3 = glm.vec4(0.0, 1, 1.0, 1);
        
        
        while(1){
            
            if(this.counter < 100){ 
                
                this.counter++;

                //----------------------- TRIÂNGULO AZUL -----------------------

                // MATRIZ IDENTIDADE onde se aplicará todas as operações
                var matIdentidade = glm.mat4(1.0);

                //------------- VETORES DA ROTAÇÃO 1 -------------


                this.revolutionVertices[0] = (this.newV1[0]);
                this.revolutionVertices[1] = (this.newV1[1]);
                this.revolutionVertices[2] = (this.newV1[2]);

                this.revolutionVertices[3] = (this.newV2[0]);
                this.revolutionVertices[4] = (this.newV2[1]);
                this.revolutionVertices[5] = (this.newV2[2]);

                this.revolutionVertices[6] = (this.newV3[0]);
                this.revolutionVertices[7] = (this.newV3[1]);
                this.revolutionVertices[8] = (this.newV3[2]);


                // Aumenta-se aqui o ângulo
                //this.angRotacao += 15;

                //------------- VETORES DA ROTAÇÃO 2 -------------
                // TRANSLAÇÃO para a origem
                var matTranslation1 = glm.mat4(1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, -this.positionX, -this.positionY, -this.positionZ, 1);
                matTranslation1 = matTranslation1['*'](matIdentidade);

                // ROTAÇÃO em torno da origem
                var rotacao = glm.angleAxis(glm.radians(this.angRotacao),glm.vec3(0, 0, 1)); //Define qual o eixo de rotação: eixo dos Z e a cada instante mudo o valor do ângulo (em graus)
                var matRotacao = glm.toMat4(rotacao);
                matRotacao = matRotacao['*'](matTranslation1);

                this.positionX = 2*Math.cos(glm.radians((this.angRotacao)));
                this.positionY = 2*Math.sin(glm.radians((this.angRotacao)));
                this.positionZ += 0.1;

                // SEGUNDA TRANSLAÇÃO 
                var matTranslation2 = glm.mat4(1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, this.positionX, this.positionY, this.positionZ, 1);
                matTranslation2 = matTranslation2['*'](matRotacao);   


                this.newV1 = matTranslation2['*'](this.newV1);
                this.newV2 = matTranslation2['*'](this.newV2);
                this.newV3 = matTranslation2['*'](this.newV3);

                this.revolutionVertices[9] = (this.newV1[0]);
                this.revolutionVertices[10] = (this.newV1[1]);
                this.revolutionVertices[11] = (this.newV1[2]);

                this.revolutionVertices[12] = (this.newV2[0]);
                this.revolutionVertices[13] = (this.newV2[1]);
                this.revolutionVertices[14] = (this.newV2[2]);

                this.revolutionVertices[15] = (this.newV3[0]);
                this.revolutionVertices[16] = (this.newV3[1]);
                this.revolutionVertices[17] = (this.newV3[2]);



                // Para unir 3 vértices de um ângulo para os 3 vértices do ângulo seguinte,
                // é necessário desenhar 6 triângulos, ou seja, 18 vértices, cada um com 3
                // componentes: X, Y e Z


                for(var i = 0; i < 3; i++){
                    // Triângulo 1
                    this.vertices[this.nciclos*18] = this.revolutionVertices[i*3];
                    this.vertices[this.nciclos*18+1] = this.revolutionVertices[i*3+1];
                    this.vertices[this.nciclos*18+2] = this.revolutionVertices[i*3+2];

                    this.vertices[this.nciclos*18+3] = this.revolutionVertices[i*3+3];
                    this.vertices[this.nciclos*18+4] = this.revolutionVertices[i*3+4];
                    this.vertices[this.nciclos*18+5] = this.revolutionVertices[i*3+5];

                    this.vertices[this.nciclos*18+6] = this.revolutionVertices[i*3+9];
                    this.vertices[this.nciclos*18+7] = this.revolutionVertices[i*3+10];
                    this.vertices[this.nciclos*18+8] = this.revolutionVertices[i*3+11];

                     // Triângulo 2
                    this.vertices[this.nciclos*18+9] = this.revolutionVertices[i*3+3];
                    this.vertices[this.nciclos*18+10] = this.revolutionVertices[i*3+4];
                    this.vertices[this.nciclos*18+11] = this.revolutionVertices[i*3+5];

                    this.vertices[this.nciclos*18+12] = this.revolutionVertices[i*3+9];
                    this.vertices[this.nciclos*18+13] = this.revolutionVertices[i*3+10];
                    this.vertices[this.nciclos*18+14] = this.revolutionVertices[i*3+11];

                    if(i == 2){
                        this.vertices[this.nciclos*18+15] = this.revolutionVertices[i*3];
                        this.vertices[this.nciclos*18+16] = this.revolutionVertices[i*3+1];
                        this.vertices[this.nciclos*18+17] = this.revolutionVertices[i*3+2];
                    }else{
                        this.vertices[this.nciclos*18+15] = this.revolutionVertices[i*3+12];
                        this.vertices[this.nciclos*18+16] = this.revolutionVertices[i*3+13];
                        this.vertices[this.nciclos*18+17] = this.revolutionVertices[i*3+14]; 
                    }



                    this.numRevolution += 6;

                    this.nciclos ++;


                }

                for(var i = 0; i < this.numRevolution*3; i = i + 3){
                    if(i%2 == 0){
                        this.colors[i] = 0;
                        this.colors[i+1] = 0;
                        this.colors[i+2] = 1;

                    }else{
                        this.colors[i] = 1;
                        this.colors[i+1] = 0;
                        this.colors[i+2] = 0;
                    }

                }

                this.numvertices =  this.numRevolution;
                
                if(this.counter >= 100){
                    break;
                }
            }
        }
            
            
        
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.vertices), this.gl.STATIC_DRAW);

        
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.colors), this.gl.STATIC_DRAW);
        
        
    }
}


</script>


<canvas id="exercise_ext" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>

<script id="myapp_ext">

class myapp_ext extends DEECapp{
    
    rotate = 0;
    
    initialize(){
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        
        this.gl.enable(this.gl.DEPTH_TEST);
        
        // Create object
        this.solid = new extSolid(this.gl);
        this.solid.setShader(this.shaderprog);

    }
    
    render(){
       
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        
        
        //------------------------- SÓLIDO A RODAR -------------------------
        
        this.rotate += 5;
        
        // TRANSLAÇÃO
        var TranslationMat = glm.translate(glm.mat4(1.0), glm.vec3(13.0,17.0,18.0));
        
        // ROTAÇÃO
        var RotMat = glm.rotate(glm.degrees(60.0), glm.vec3(0.0,1.0,0.0));
        var RotMat2 = glm.rotate(glm.radians(this.rotate), glm.vec3(1.0,0.0,0.0));
        
        var myModelMatrix = TranslationMat['*'](RotMat['*'](RotMat2));

        // TRANSFORMAÇÃO DE VISTA/PERSPETIVA
        var ViewMatrix = glm.lookAt(glm.vec3(4,3,3), glm.vec3(9.5,9.5,9.5), glm.vec3(0,1,0));
        
        // PROJEÇÃO - FoV, aspectratio, nearplane, farplane
        var ProjectionMatrix = glm.perspective(glm.radians(45.0), 1.0, 0.1, 100.0);
        
        this.solid.setModelTransformation(myModelMatrix);
        
        // Utilizar o método drawit fornecido
        this.solid.drawit(ViewMatrix, ProjectionMatrix);

    }
}

var app_ext = new myapp_ext('exercise_ext');

app_ext.run();

</script>

<h1>Compound Shapes</h1>

<p>
    Using the shapes created earlier, several objects were made to include in the scene, such as: tree, pine tree, house, cloud, bush, bench, bowl, and car.
</p>

<h2>Trees</h2>

<p>
    The trees are simple, as they only require a sphere and a cylinder for the trunk. The sphere has a more "square" appearance to visualize the texture and simulate leaves.
</p>

<p>
    The pine trees were designed with a cylinder for the trunk and three cones for the foliage.
</p>

In [18]:
%%html

<script>

class arvore extends CGRAobject{
    constructor(glcontext){
        super(glcontext);
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        var fragsrcT = document.getElementById("my-fragment-shaderT").text;
        var vertsrcT = document.getElementById("my-vertex-shaderT").text;
        
        var fragsrcL = document.getElementById("my-fragment-shaderL").text;
        var vertsrcL = document.getElementById("my-vertex-shaderL").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        this.shaderprogT = new DEECshader(this.gl);
        this.shaderprogT.srcShaders(vertsrcT,fragsrcT);
        
        this.shaderprogL = new DEECshader(this.gl);
        this.shaderprogL.srcShaders(vertsrcL,fragsrcL);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("Images/folhas.jpg");
        
        this.folhas = new esferaL(this.gl,10,21/255,62/255,42/255,1);
        this.folhas.settexture(this.texture);
        this.folhas.setShader(this.shaderprogL);
        
        
        this.texture2 = new CGRAtexture(this.gl);
        this.texture2.load("Images/tronco2.jpg");
    
        // ntriangulos, red, green, blue, samecolor
        this.tronco = new cilindroL(this.gl,30, 0.6, 0.4, 0.2, 1);
        this.tronco.settexture(this.texture2);
        this.tronco.setShader(this.shaderprogL); 
        
        
    }
    
    
    drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2){
        //------------------------ TRONCO ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0,0,0));
        
        var ScalingMat = glm.scale(glm.vec3(0.6, 0.6, 3));
        
        var RotMat = glm.rotate(glm.mat4(1.0),glm.degrees(0),glm.vec3(0,1,0));
        
        var myModelMatrix = RotMat['*'](TranslationMat['*'](ScalingMat));
        
        this.tronco.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.tronco.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        //------------------------ FOLHAS ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0,0,0));
        
        var ScalingMat = glm.scale(glm.vec3(1.0,1.0,1.0));
        
        var RotMat = glm.rotate(glm.mat4(1.0),glm.radians(-90),glm.vec3(1,0,0));
        
        var myModelMatrix = RotMat['*']((ScalingMat));
        
        this.folhas.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.folhas.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);     


    }

}


</script>


<canvas id="myCanvasArvore" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp">

class myappArvore extends DEECapp{
    
    angRotacao = 0;
    
    initialize(){
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;

        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        
        this.gl.enable(this.gl.DEPTH_TEST);
        
        // Create cube object
        this.myarvore = new arvore(this.gl);
        this.myarvore.setShader(this.shaderprog);

    }
    
    render(){
       
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        
        this.angRotacao = this.angRotacao + 3;
        
        
        // TRANSLAÇÃO
        var TranslationMat = glm.translate(glm.mat4(1.0), glm.vec3(12.0,10.0,10.0));
        
        // ROTAÇÃO
        var RotMat = glm.rotate(glm.radians(this.angRotacao), glm.vec3(0.0,0.0,1.0));

        var myModelMatrix = TranslationMat['*'](RotMat);
        

        // TRANSFORMAÇÃO DE VISTA/PERSPETIVA
        var ViewMatrix = glm.lookAt(
            glm.vec3(4,4,4),       // cameraPosition
            glm.vec3(9.5,9.5,9.5), // cameraTarget
            glm.vec3(1,1,0)        // upVector
        );
        
        // PROJEÇÃO - FoV, aspectratio, nearplane, farplane
        var ProjectionMatrix = glm.perspective(glm.radians(45.0), 1.0, 0.1, 100.0);
        
        // Computar a transformação aplicada a cada vértice do modelo
        var MVPmatrix = ProjectionMatrix['*'](ViewMatrix['*'](myModelMatrix));
        
        this.myarvore.setModelTransformation(myModelMatrix);
        
        // Utilizar o método drawit fornecido
        this.myarvore.drawit(ViewMatrix, ProjectionMatrix, 1, 0, 0);

        
    }
}

var appArvore = new myappArvore('myCanvasArvore');

appArvore.run();
</script>


In [19]:
%%html

<script>

class pinheiro extends CGRAobject{
    constructor(glcontext){
        super(glcontext);
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        var fragsrcT = document.getElementById("my-fragment-shaderT").text;
        var vertsrcT = document.getElementById("my-vertex-shaderT").text;
        
        var fragsrcL = document.getElementById("my-fragment-shaderL").text;
        var vertsrcL = document.getElementById("my-vertex-shaderL").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        this.shaderprogT = new DEECshader(this.gl);
        this.shaderprogT.srcShaders(vertsrcT,fragsrcT);
        
        this.shaderprogL = new DEECshader(this.gl);
        this.shaderprogL.srcShaders(vertsrcL,fragsrcL);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("Images/texturaNatal.jpg");

        // ntriangulos, red, green, blue, samecolor
        this.folhas = new coneL(this.gl,30,21/255,62/255,42/255,1);
        this.folhas.settexture(this.texture);
        this.folhas.setShader(this.shaderprogL);
        this.folhas1 = new coneL(this.gl,26,3/255,78/255,53/255,1);
        this.folhas1.settexture(this.texture);
        this.folhas1.setShader(this.shaderprogL);
        this.folhas2 = new coneL(this.gl,30,33/255,98/255,66/255,1);
        this.folhas2.settexture(this.texture);
        this.folhas2.setShader(this.shaderprogL);
        
        this.texture2 = new CGRAtexture(this.gl);
        this.texture2.load("Images/tronco2.jpg");
    
        // ntriangulos, red, green, blue, samecolor
        this.tronco = new cilindroL(this.gl,30, 0.6, 0.4, 0.2, 1);
        this.tronco.settexture(this.texture2);
        this.tronco.setShader(this.shaderprogL);        
        
    }
    
    
    
    drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2){
        //------------------------ TRONCO ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0,0,0));
        
        var ScalingMat = glm.scale(glm.vec3(0.6, 0.6, 1.5));
        
        var RotMat = glm.rotate(glm.mat4(1.0),glm.degrees(0),glm.vec3(0,1,0));
        
        var myModelMatrix = RotMat['*'](TranslationMat['*'](ScalingMat));
        
        this.tronco.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.tronco.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        //------------------------ FOLHAS ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0,0,0));
        
        var ScalingMat = glm.scale(glm.vec3(2.5,2.5,1.75));
        
        var RotMat = glm.rotate(glm.mat4(1.0),glm.radians(-180),glm.vec3(1,0,0));
        
        var myModelMatrix = RotMat['*']((ScalingMat));
        
        this.folhas.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.folhas.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        //------------------ Folha1 ------------------ 
        var TranslationMat1 = glm.translate(glm.mat4(1.0),glm.vec3(0.0,0.0,-1.25));
        
        var ScalingMat1 = glm.scale(glm.vec3(2.0,2.0,1.5));
        
        var RotMat = glm.rotate(glm.mat4(1.0),glm.radians(-180),glm.vec3(1,0,0));
        
        var myModelMatrix = TranslationMat1['*'](RotMat['*'](ScalingMat1));
        
        
        this.folhas1.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.folhas1.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
         //------------------ Folha2 ------------------ 
        var TranslationMat2 = glm.translate(glm.mat4(1.0),glm.vec3(0.0,0.0,-2.5));
        
        var ScalingMat2 = glm.scale(glm.vec3(1.5,1.5,1.25));
        
        var RotMat = glm.rotate(glm.mat4(1.0),glm.radians(-180),glm.vec3(1,0,0));
        
        var myModelMatrix = TranslationMat2['*'](RotMat['*'](ScalingMat2));
        
        
        this.folhas2.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.folhas2.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        

    }

}


</script>


<canvas id="myCanvasPinheiro" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp">

class myappPinheiro extends DEECapp{
    
    angRotacao = 0;
    
    initialize(){
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;

        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        
        this.gl.enable(this.gl.DEPTH_TEST);
        
        // Create cube object
        this.mypinheiro = new pinheiro(this.gl);
        this.mypinheiro.setShader(this.shaderprog);

    }
    
    render(){
       
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        
        this.angRotacao = this.angRotacao + 3;
        
        
        // TRANSLAÇÃO
        var TranslationMat = glm.translate(glm.mat4(1.0), glm.vec3(12.0,10.0,10.0));
        
        // ROTAÇÃO
        var myRotAxis = glm.vec3(0.0,1.0,0.0);
        var RotMat = glm.rotate(glm.radians(this.angRotacao),myRotAxis);

        // var myModelMatrix = TranslationMat['*'](RotMat['*'](ScalingMat));
        var myModelMatrix = TranslationMat['*'](RotMat);
        // var myModelMatrix = TranslationMat;
        

        // TRANSFORMAÇÃO DE VISTA/PERSPETIVA
        var ViewMatrix = glm.lookAt(
            glm.vec3(1,1,4),       // cameraPosition
            glm.vec3(9.5,9.5,9.5), // cameraTarget
            glm.vec3(1,1,0)        // upVector
        );
        
        // PROJEÇÃO - FoV, aspectratio, nearplane, farplane
        var ProjectionMatrix = glm.perspective(glm.radians(45.0), 1.0, 0.2, 100.0);
        
        // Computar a transformação aplicada a cada vértice do modelo
        var MVPmatrix = ProjectionMatrix['*'](ViewMatrix['*'](myModelMatrix));
        
        this.mypinheiro.setModelTransformation(myModelMatrix);
        
        // Utilizar o método drawit fornecido
        this.mypinheiro.drawit(ViewMatrix, ProjectionMatrix, 1, 0, 0);

        
    }
}

var appPinheiro = new myappPinheiro('myCanvasPinheiro');

appPinheiro.run();
</script>


<h2>House</h2>

<p>
    The house uses two cubes for its structure and a cone (with four sides) to form a square pyramid.
</p>

In [20]:
%%html

<script>

class casa extends CGRAobject{
    constructor(glcontext,rp,gp,bp,rt,gt,bt){
        super(glcontext);
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        var fragsrcL = document.getElementById("my-fragment-shaderL").text;
        var vertsrcL = document.getElementById("my-vertex-shaderL").text;
        

        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        this.shaderprogL = new DEECshader(this.gl);
        this.shaderprogL.srcShaders(vertsrcL,fragsrcL);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("Images/brick.jpg");
        
        this.texture2 = new CGRAtexture(this.gl);
        this.texture2.load("Images/telhas.jpg");
        
        this.texture3 = new CGRAtexture(this.gl);
        this.texture3.load("Images/tronco2.jpg");
        
        
        // ntriangulos, red, green, blue, samecolor
        this.paredes = new cuboL(this.gl,0.8,0.8,0.8,1, 0);
        this.paredes.settexture(this.texture);
        this.paredes.setShader(this.shaderprogL);
        
        this.teto = new coneL(this.gl,4,160/255, 82/255, 45/255,1)
        this.teto.settexture(this.texture2);
        this.teto.setShader(this.shaderprogL);
        
        
        this.porta = new cuboL(this.gl,139/255,69/255,19/255,1, 0);
        this.porta.settexture(this.texture3);
        this.porta.setShader(this.shaderprogL);
        
        
    }
    
    
    
    drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2){
        //------------------------ PAREDES ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0,0,4));
        
        var ScalingMat = glm.scale(glm.vec3(4, 4, 4));
        
        var RotMat = glm.rotate(glm.mat4(1.0),glm.radians(45),glm.vec3(0,0,1));
        
        var myModelMatrix = RotMat['*'](TranslationMat['*'](ScalingMat));
        
        this.paredes.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.paredes.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        //------------------------ TETO ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0,0,-2.3));
        
        var ScalingMat = glm.scale(glm.vec3(3.2,3.2,1));
        
        var RotMat = glm.rotate(glm.mat4(1.0),glm.radians(-180),glm.vec3(1,0,0));
        
        var myModelMatrix = RotMat['*'](TranslationMat['*'](ScalingMat));
        
        this.teto.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.teto.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        //------------------------ PORTA ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(-1.2,0,4.5));
        
        var ScalingMat = glm.scale(glm.vec3(2,1,2));
        
        var RotMat = glm.rotate(glm.mat4(1.0),glm.radians(45),glm.vec3(0,0,1));
        
        var myModelMatrix = RotMat['*'](TranslationMat['*'](ScalingMat));
        
        this.porta.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.porta.drawit(ViewMatrix,ProjectionMatrix);

    }

}


</script>


<canvas id="myCanvasCasa" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp">

class myappCasa extends DEECapp{
    
    angRotacao = 0;
    
    initialize(){
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;

        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        
        this.gl.enable(this.gl.DEPTH_TEST);
        
        // Create cube object
        this.mycasa = new casa(this.gl,0,1,127/255,65/255,85/255,90/255);
        this.mycasa.setShader(this.shaderprog);

    }
    
    render(){
       
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        
        this.angRotacao = this.angRotacao + 3;
        
        
        // TRANSLAÇÃO
        var TranslationMat = glm.translate(glm.mat4(1.0), glm.vec3(12.0,12.0,8.0));
        
        // ROTAÇÃO
        var RotMat = glm.rotate(glm.radians(this.angRotacao), glm.vec3(0.0,0.0,1.0));

        var myModelMatrix = TranslationMat['*'](RotMat);
        

        // TRANSFORMAÇÃO DE VISTA/PERSPETIVA
        var ViewMatrix = glm.lookAt(
            glm.vec3(1,1,4),       // cameraPosition
            glm.vec3(9.5,9.5,9.5), // cameraTarget
            glm.vec3(1,1,0)        // upVector
        );
        
        // PROJEÇÃO - FoV, aspectratio, nearplane, farplane
        var ProjectionMatrix = glm.perspective(glm.radians(45.0), 1.0, 0.2, 100.0);
        
        // Computar a transformação aplicada a cada vértice do modelo
        var MVPmatrix = ProjectionMatrix['*'](ViewMatrix['*'](myModelMatrix));
        
        this.mycasa.setModelTransformation(myModelMatrix);
        
        // Utilizar o método drawit fornecido
        this.mycasa.drawit(ViewMatrix, ProjectionMatrix, 1, 0, 0);

        
    }
}

var appCasa = new myappCasa('myCanvasCasa');

appCasa.run();
</script>


<h2>Cloud</h2>

<p>
    The cloud consists of just a set of spheres.
</p>

In [21]:
%%html

<script>

class nuvem extends CGRAobject{
    constructor(glcontext){
        super(glcontext);
        
        var fragsrcL = document.getElementById("my-fragment-shaderL").text;
        var vertsrcL = document.getElementById("my-vertex-shaderL").text;
        
        this.shaderprogL = new DEECshader(this.gl);
        this.shaderprogL.srcShaders(vertsrcL,fragsrcL);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("Images/nuvens2.jpg");
        
        // ntriangulos, red, green, blue, samecolor
        this.esfera = new esferaL(this.gl,30, 1.0, 1.0, 1.0, 1);
        this.esfera.settexture(this.texture);
        this.esfera.setShader(this.shaderprogL);
        
        
    }
    
    
    drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2){
        
        //------------------------ ESFERA 1 ------------------------
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0, 0.4, 0));
        
        var ScalingMat = glm.scale(glm.vec3(1.1,1.1,0.8));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.esfera.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.esfera.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2); 
        
    
        //------------------------ ESFERA 2 ------------------------
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(2, 0, 0));
        
        var ScalingMat = glm.scale(glm.vec3(0.6, 0.6, 0.6));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.esfera.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.esfera.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        
        //------------------------ ESFERA 3 ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(-2, 0, 0));
        
        var ScalingMat = glm.scale(glm.vec3(0.8, 0.8, 0.65));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.esfera.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.esfera.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        //------------------------ ESFERA 4 ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(-1, 2, 0));
        
        var ScalingMat = glm.scale(glm.vec3(0.6, 0.6, 0.5));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.esfera.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.esfera.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);

    }

}


</script>


<canvas id="myCanvasNuvem" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp">

class myappNuvem extends DEECapp{
    
    angRotacao = 0;
    
    initialize(){
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;

        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        this.gl.clearColor(0.67,0.923,0.99,1.0);
       
        this.gl.enable(this.gl.DEPTH_TEST);
        
        // Create cube object
        this.mynuvem = new nuvem(this.gl);
        this.mynuvem.setShader(this.shaderprog);

    }
    
    render(){
       
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        
        this.angRotacao = this.angRotacao + 3;
        
        
        // TRANSLAÇÃO
        var TranslationMat = glm.translate(glm.mat4(1.0), glm.vec3(0, 0, 0));
        
        // ROTAÇÃO
        var RotMat = glm.rotate(glm.radians(this.angRotacao), glm.vec3(0.0,1.0,0.0));

        // var myModelMatrix = TranslationMat['*'](RotMat['*'](ScalingMat));
        var myModelMatrix = TranslationMat['*'](RotMat);   

        // TRANSFORMAÇÃO DE VISTA/PERSPETIVA
        var ViewMatrix = glm.lookAt(
            glm.vec3(6,6,6),       // cameraPosition
            glm.vec3(0, 0, 0), // cameraTarget
            glm.vec3(0,1,0)        // upVector
        );
        
        // PROJEÇÃO - FoV, aspectratio, nearplane, farplane
        var ProjectionMatrix = glm.perspective(glm.radians(45.0), 1.0, 0.1, 100.0);
        
        this.mynuvem.setModelTransformation(myModelMatrix);
        
        // Utilizar o método drawit fornecido
        this.mynuvem.drawit(ViewMatrix, ProjectionMatrix, 1, 0, 0);

        
    }
}

var appNuvem = new myappNuvem('myCanvasNuvem');

appNuvem.run();
</script>


<h2>Bush</h2>

<p>
    The bush consists of just a set of spheres.
</p>

In [22]:
%%html

<script>

class arbusto extends CGRAobject{
    constructor(glcontext,r,g,b){
        super(glcontext);
        
        var fragsrcL = document.getElementById("my-fragment-shaderL").text;
        var vertsrcL = document.getElementById("my-vertex-shaderL").text;
        
        this.shaderprogL = new DEECshader(this.gl);
        this.shaderprogL.srcShaders(vertsrcL,fragsrcL);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("Images/folhas.jpg");
        
        // ntriangulos, red, green, blue, samecolor
        this.esfera = new esferaL(this.gl,15, r, g, b, 1);
        this.esfera.settexture(this.texture);
        this.esfera.setShader(this.shaderprogL);
        
    }
    
    
    
    drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2){
        
        //------------------------ ESFERA 1 ------------------------
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0, 0.4, 0));
        
        var ScalingMat = glm.scale(glm.vec3(1.1,1.1,1.1));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.esfera.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.esfera.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2); 
        
    
        //------------------------ ESFERA 2 ------------------------
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(2, 0, 0));
        
        var ScalingMat = glm.scale(glm.vec3(0.7, 0.6, 0.8));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.esfera.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.esfera.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        
        //------------------------ ESFERA 3 ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(-2, 0, 0));
        
        var ScalingMat = glm.scale(glm.vec3(0.8, 0.8, 0.9));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.esfera.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.esfera.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        //------------------------ ESFERA 4 ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(-1, 2, 0));
        
        var ScalingMat = glm.scale(glm.vec3(0.7, 0.6, 0.8));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.esfera.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.esfera.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        //------------------------ ESFERA 5 ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(1, 2, 0));
        
        var ScalingMat = glm.scale(glm.vec3(0.7, 0.6, 0.8));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.esfera.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.esfera.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        //------------------------ ESFERA 6 ------------------------
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0, -0.9, 0));
        
        var ScalingMat = glm.scale(glm.vec3(1.6,0.5,0.9));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.esfera.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.esfera.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2); 

    }

}


</script>


<canvas id="myCanvasArbusto" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp">

class myappArbusto extends DEECapp{
    
    angRotacao = 0;
    
    initialize(){
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;

        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        
        this.gl.enable(this.gl.DEPTH_TEST);
        
        // Create cube object
        this.myarbusto = new arbusto(this.gl,21/255,62/255,42/255);
        this.myarbusto.setShader(this.shaderprog);

    }
    
    render(){
       
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        
        this.angRotacao = this.angRotacao + 3;
        
        
        // TRANSLAÇÃO
        var TranslationMat = glm.translate(glm.mat4(1.0), glm.vec3(0, 0, 0));
        
        // ROTAÇÃO
        var myRotAxis = glm.vec3(0.0,1.0,0.0);
        var RotMat = glm.rotate(glm.radians(this.angRotacao),myRotAxis);

        // var myModelMatrix = TranslationMat['*'](RotMat['*'](ScalingMat));
        var myModelMatrix = TranslationMat['*'](RotMat);
        //var myModelMatrix = TranslationMat;
        

        // TRANSFORMAÇÃO DE VISTA/PERSPETIVA
        var ViewMatrix = glm.lookAt(
            glm.vec3(6,6,6),       // cameraPosition
            glm.vec3(0, 0, 0), // cameraTarget
            glm.vec3(0,1,0)        // upVector
        );
        
        // PROJEÇÃO - FoV, aspectratio, nearplane, farplane
        var ProjectionMatrix = glm.perspective(glm.radians(45.0), 1.0, 0.1, 100.0);
        
        // Computar a transformação aplicada a cada vértice do modelo
        var MVPmatrix = ProjectionMatrix['*'](ViewMatrix['*'](myModelMatrix));
        
        this.myarbusto.setModelTransformation(myModelMatrix);
        
        // Utilizar o método drawit fornecido
        this.myarbusto.drawit(ViewMatrix, ProjectionMatrix, 1, 0, 0);

        
    }
}

var appArbusto = new myappArbusto('myCanvasArbusto');

appArbusto.run();
</script>


<h2>Person</h2>

<p>
    This person was created to decorate the bench, using a sphere for the head, a cone for the neck, and a cylinder for the torso.
</p>

In [23]:
%%html

<script>

class pessoa extends CGRAobject{
    constructor(glcontext, r1, g1, b1, r2, g2, b2, pessoa){
        super(glcontext);
        
        var fragsrcL = document.getElementById("my-fragment-shaderL").text;
        var vertsrcL = document.getElementById("my-vertex-shaderL").text;        

        this.shaderprogL = new DEECshader(this.gl);
        this.shaderprogL.srcShaders(vertsrcL,fragsrcL);
        
        if(pessoa == 1){
            this.texture = new CGRAtexture(this.gl);
            this.texture.load("Images/Camisola1.png");

            this.texture2 = new CGRAtexture(this.gl);
            this.texture2.load("Images/Pessoa1.png");
        }else if(pessoa == 2){
            this.texture = new CGRAtexture(this.gl);
            this.texture.load("Images/Camisola2.jpg");

            this.texture2 = new CGRAtexture(this.gl);
            this.texture2.load("Images/Pessoa2.jpg");
        }else{
            this.texture = new CGRAtexture(this.gl);
            this.texture.load("Images/Camisola3.png");

            this.texture2 = new CGRAtexture(this.gl);
            this.texture2.load("Images/Pessoa3.jpg");
        }
        
        // ntriangulos, red, green, blue, samecolor
        this.esfera = new esferaL(this.gl,30, r1, g1, b1, 1);
        this.esfera.settexture(this.texture2);
        this.esfera.setShader(this.shaderprogL);
                
        // ntriangulos, red, green, blue, samecolor
        this.cone = new coneL(this.gl,30, r1, g1, b1, 1);
        this.cone.settexture(this.texture2);
        this.cone.setShader(this.shaderprogL);
        
        // red, green, blue, samecolor
        this.cilindro = new cilindroL(this.gl, 30, r2, g2, b2, 1);
        this.cilindro.settexture(this.texture);
        this.cilindro.setShader(this.shaderprogL);
        
        
    } 
    
    drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2){
        
        //------------------------ ESFERA ------------------------
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0, 0.4, 0));
        
        var myRotAxis = glm.vec3(0.0,1.0,0.0);
        var RotMat = glm.rotate(glm.radians(0),myRotAxis);
        
        var ScalingMat = glm.scale(glm.vec3(0.5,0.5,0.5));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.esfera.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.esfera.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2); 
        
    
        //------------------------ CONE ------------------------
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0, -1, 0));
        
        var myRotAxis = glm.vec3(1.0,0.0,0.0);
        var RotMat = glm.rotate(glm.radians(-90),myRotAxis);
        
        var ScalingMat = glm.scale(glm.vec3(0.7, 0.7, 0.7));
        
        var myModelMatrix = TranslationMat['*'](RotMat['*'](ScalingMat));
        
        this.cone.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.cone.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        
        //------------------------ CILINDRO ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0, -2.3, 0));
        
        var ScalingMat = glm.scale(glm.vec3(1.0, 2.0, 0.7));
        
        var myModelMatrix = TranslationMat['*'](RotMat['*'](ScalingMat));
        
        this.cilindro.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.cilindro.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);

    }

}


</script>


<canvas id="myCanvasPessoa" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp">

class myappPessoa extends DEECapp{
    
    angRotacao = 0;
    
    initialize(){
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;

        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        
        this.gl.enable(this.gl.DEPTH_TEST);
        
        // Create cube object

        this.mypessoa = new pessoa(this.gl, 0.7, 0.3, 0.25, 0.3, 0.35, 0.75, 1);
        this.mypessoa.setShader(this.shaderprog);

    }
    
    render(){
       
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        
        this.angRotacao = this.angRotacao + 3;
        
        
        // TRANSLAÇÃO
        var TranslationMat = glm.translate(glm.mat4(1.0), glm.vec3(0, 0, 0));
        
        // ROTAÇÃO
        var myRotAxis = glm.vec3(0.0,1.0,0.0);
        var RotMat = glm.rotate(glm.radians(this.angRotacao),myRotAxis);

        // var myModelMatrix = TranslationMat['*'](RotMat['*'](ScalingMat));
        var myModelMatrix = TranslationMat['*'](RotMat);
        //var myModelMatrix = TranslationMat;
        

        // TRANSFORMAÇÃO DE VISTA/PERSPETIVA
        var ViewMatrix = glm.lookAt(
            glm.vec3(7,7,7),       // cameraPosition
            glm.vec3(0, 0, 0), // cameraTarget
            glm.vec3(0,1,0)        // upVector
        );
        
        // PROJEÇÃO - FoV, aspectratio, nearplane, farplane
        var ProjectionMatrix = glm.perspective(glm.radians(45.0), 1.0, 0.1, 100.0);
        
        // Computar a transformação aplicada a cada vértice do modelo
        var MVPmatrix = ProjectionMatrix['*'](ViewMatrix['*'](myModelMatrix));
        
        this.mypessoa.setModelTransformation(myModelMatrix);
        
        // Utilizar o método drawit fornecido
        this.mypessoa.drawit(ViewMatrix, ProjectionMatrix, 0, 0, 0);

        
    }
}

var appPessoa = new myappPessoa('myCanvasPessoa');

appPessoa.run();
</script>

<h2>Bench</h2>

<p>
    The bench was created using cubes and cylinders for its shape, and people were also added.
</p>

In [24]:
%%html

<script>

class bancada extends CGRAobject{
    constructor(glcontext,r,g,b, bancada){
        super(glcontext);
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        var fragsrcL = document.getElementById("my-fragment-shaderL").text;
        var vertsrcL = document.getElementById("my-vertex-shaderL").text;
        
        this.shaderprogL = new DEECshader(this.gl);
        this.shaderprogL.srcShaders(vertsrcL,fragsrcL);
        
        if(bancada == 1){
            this.texture = new CGRAtexture(this.gl);
            this.texture.load("Images/azul.jpg");
        }else{
            this.texture = new CGRAtexture(this.gl);
            this.texture.load("Images/vermelho.jpg");
        }

        this.texture2 = new CGRAtexture(this.gl);
        this.texture2.load("Images/cinza.jpg");
        
        // red, green, blue, samecolor
        this.cubo = new cuboL(this.gl, r,g,b, 1, 0);
        this.cubo.settexture(this.texture);
        this.cubo.setShader(this.shaderprogL);
        
        // red, green, blue, samecolor
        this.cilindro = new cilindroL(this.gl, 30, 0.75, 0.75, 0.75, 1);
        this.cilindro.settexture(this.texture2);
        this.cilindro.setShader(this.shaderprogL);
        
        // RGB - cabeça, camisola
        this.pessoa1 = new pessoa(this.gl, 0.7, 0.3, 0.25, 0.95, 0.95, 0.15, 1);
        this.pessoa1.setShader(this.shaderprog);
        
        this.pessoa2 = new pessoa(this.gl, 0.9, 0.8, 0.8, 0.9, 0.25, 0.55, 2);
        this.pessoa2.setShader(this.shaderprog);
        
        this.pessoa3 = new pessoa(this.gl, 0.7, 0.5, 0.7, 0.25, 0.65, 0.65, 3);
        this.pessoa3.setShader(this.shaderprog);
        
    }
    
    
    drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2){
        
        //------------------------ LATERAIS ------------------------
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0, 0, 7.2));
        
        var myRotAxis = glm.vec3(1.0,0.0,0.0);
        var RotMat = glm.rotate(glm.radians(-90),myRotAxis);
        
        var ScalingMat = glm.scale(glm.vec3(3.0,0.5,3.0));
        
        var myModelMatrix = TranslationMat['*'](RotMat['*'](ScalingMat));
        
        this.cilindro.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.cilindro.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2); 
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0, 0, -7.2));
        
        var myModelMatrix = TranslationMat['*'](RotMat['*'](ScalingMat));
        
        this.cilindro.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.cilindro.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        //------------------------ BAIXO ------------------------
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0, 1, 0));
        
        var ScalingMat = glm.scale(glm.vec3(7.4, 1, 17));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.cubo.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.cubo.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        
        //------------------------ TRÁS ------------------------
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(-2.8, 5.1, 0));
        
        var myRotAxis = glm.vec3(1.0,0.0,0.0);
        var RotMat = glm.rotate(glm.radians(90),myRotAxis);
        
        var ScalingMat = glm.scale(glm.vec3(0.5,7.3,2.6));
        
        var myModelMatrix = TranslationMat['*'](RotMat['*'](ScalingMat));
        
        this.cilindro.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.cilindro.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
    
        //------------------------ TETO ------------------------
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0, 6, 0));
        
        var myRotAxis = glm.vec3(0.0,0.0,1.0);
        var RotMat = glm.rotate(glm.radians(9),myRotAxis);
        
        var ScalingMat = glm.scale(glm.vec3(8, 1.2, 20));
        
        var myModelMatrix = TranslationMat['*'](RotMat['*'](ScalingMat));
        //var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.cubo.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.cubo.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        //------------------------ FRENTE ------------------------
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(3.2, 0.8, 0));
        
        var ScalingMat = glm.scale(glm.vec3(0.5, 2, 19));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.cubo.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.cubo.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        var TranslationPessoa = [];

        //------------------------ PESSOAS ------------------------
        
        var ScalingMat = glm.scale(glm.vec3(0.4, 0.4, 0.4));
       
        //---------------------------- FILA 1 ----------------------------
        
        //------------------------ PESSOA 1 ------------------------
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(-1, 5, -5));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.pessoa1.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.pessoa1.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        
        //------------------------ PESSOA 2 ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(-1, 5, -3));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.pessoa1.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.pessoa1.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        //------------------------ PESSOA 3 ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(-1, 5, -1));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.pessoa2.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.pessoa2.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        //------------------------ PESSOA 4 ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(-1, 5, 1));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.pessoa3.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.pessoa3.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        //------------------------ PESSOA 5 ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(-1, 5, 3));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.pessoa2.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.pessoa2.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        //------------------------ PESSOA 6 ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(-1, 5, 5));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.pessoa1.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.pessoa1.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);

        //---------------------------- FILA 2 ----------------------------
        
        //------------------------ PESSOA 1 ------------------------
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0, 4, -5));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.pessoa3.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.pessoa3.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        
        //------------------------ PESSOA 2 ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0, 4, -3));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.pessoa3.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.pessoa3.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        //------------------------ PESSOA 3 ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0, 4, -1));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.pessoa2.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.pessoa2.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        //------------------------ PESSOA 4 ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0, 4, 1));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.pessoa2.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.pessoa2.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        //------------------------ PESSOA 5 ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0, 4, 3));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.pessoa3.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.pessoa3.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        //------------------------ PESSOA 6 ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0, 4, 5));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.pessoa1.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.pessoa1.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        //---------------------------- FILA 3 ----------------------------
        
        //------------------------ PESSOA 1 ------------------------
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(1, 3, -5));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.pessoa2.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.pessoa2.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        
        //------------------------ PESSOA 2 ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(1, 3, -3));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.pessoa1.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.pessoa1.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        //------------------------ PESSOA 3 ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(1, 3, -1));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.pessoa3.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.pessoa3.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        //------------------------ PESSOA 4 ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(1, 3, 1));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.pessoa1.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.pessoa1.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        //------------------------ PESSOA 5 ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(1, 3, 3));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.pessoa3.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.pessoa3.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        //------------------------ PESSOA 6 ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(1, 3, 5));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.pessoa2.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.pessoa2.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
    }

}


</script>


<canvas id="myCanvasBancada" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp">

class myappBancada extends DEECapp{
    
    angRotacao = 0;
    
    initialize(){
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;

        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        
        this.gl.enable(this.gl.DEPTH_TEST);
        
        // Create cube object
        this.mybancada = new bancada(this.gl,6,0,0.7, 1, 1);
        this.mybancada.setShader(this.shaderprog);

    }
    
    render(){
       
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        
        this.angRotacao = this.angRotacao + 3;
        
        
        // TRANSLAÇÃO
        var TranslationMat = glm.translate(glm.mat4(1.0), glm.vec3(0, 0, 0));
        
        // ROTAÇÃO
        var myRotAxis = glm.vec3(0.0,1.0,0.0);
        var RotMat = glm.rotate(glm.radians(this.angRotacao),myRotAxis);
        //var RotMat = glm.rotate(glm.radians(-40),myRotAxis);

        // var myModelMatrix = TranslationMat['*'](RotMat['*'](ScalingMat));
        var myModelMatrix = TranslationMat['*'](RotMat);
        // var myModelMatrix = TranslationMat;
        

        // TRANSFORMAÇÃO DE VISTA/PERSPETIVA
        var ViewMatrix = glm.lookAt(
            glm.vec3(15, 10, 15),       // cameraPosition
            glm.vec3(0, 0, 0), // cameraTarget
            glm.vec3(0,1,0)        // upVector
        );
        
        // PROJEÇÃO - FoV, aspectratio, nearplane, farplane
        var ProjectionMatrix = glm.perspective(glm.radians(45.0), 1.0, 0.1, 100.0);
        
        // Computar a transformação aplicada a cada vértice do modelo
        var MVPmatrix = ProjectionMatrix['*'](ViewMatrix['*'](myModelMatrix));
        
        this.mybancada.setModelTransformation(myModelMatrix);
        
        // Utilizar o método drawit fornecido
        this.mybancada.drawit(ViewMatrix, ProjectionMatrix, 1, 0, 0);

        
    }
}

var appBancada = new myappBancada('myCanvasBancada');

appBancada.run();
</script>

<h2>Trophy</h2>

<p>
    The trophy was created using a cone and a cylinder for the base, and a cylinder and a sphere for the top.
</p>

In [25]:
%%html

<script>

class taca extends CGRAobject{
    constructor(glcontext){
        super(glcontext);
        
        var fragsrcL = document.getElementById("my-fragment-shaderL").text;
        var vertsrcL = document.getElementById("my-vertex-shaderL").text;
        
        this.shaderprogL = new DEECshader(this.gl);
        this.shaderprogL.srcShaders(vertsrcL,fragsrcL);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("Images/gold2.jpg");
        
        // ntriangulos, red, green, blue, samecolor
        this.esfera = new esferaL(this.gl,30, 1, 215/255, 0, 1);
        this.esfera.settexture(this.texture);
        this.esfera.setShader(this.shaderprogL);
                
        // ntriangulos, red, green, blue, samecolor
        this.cone = new coneL(this.gl,30, 1, 215/255, 0, 1);
        this.cone.settexture(this.texture);
        this.cone.setShader(this.shaderprogL);
        
        // red, green, blue, samecolor
        this.cilindro = new cilindroL(this.gl, 30, 1, 215/255, 0, 1);
        this.cilindro.settexture(this.texture);
        this.cilindro.setShader(this.shaderprogL);
        
        this.texture2 = new CGRAtexture(this.gl);
        this.texture2.load("Images/gold.jpg");
        
        this.cilindro2 = new cilindroL(this.gl, 30, 238/255, 188/255, 29/255, 1);
        this.cilindro2.settexture(this.texture2);
        this.cilindro2.setShader(this.shaderprogL);
        
        
    }
    
    
    
    drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2){
        
        //------------------------ ESFERA ------------------------
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0, 0.8, 0));
        
        var myRotAxis = glm.vec3(0.0,1.0,0.0);
        var RotMat = glm.rotate(glm.radians(0),myRotAxis);
        
        var ScalingMat = glm.scale(glm.vec3(0.6,0.6,0.6));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.esfera.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.esfera.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2); 
        
    
        //------------------------ CONE ------------------------
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0, -1.5, 0));
        
        var myRotAxis = glm.vec3(1.0,0.0,0.0);
        var RotMat = glm.rotate(glm.radians(-90),myRotAxis);
        
        var ScalingMat = glm.scale(glm.vec3(1.2, 1.2, 0.4));
        
        var myModelMatrix = TranslationMat['*'](RotMat['*'](ScalingMat));
        
        this.cone.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.cone.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        
        //------------------------ CILINDRO ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0, -0.9, 0));
        
        var ScalingMat = glm.scale(glm.vec3(0.3, 0.3, 0.3));
        
        var myModelMatrix = TranslationMat['*'](RotMat['*'](ScalingMat));
        
        this.cilindro.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.cilindro.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
         //------------------------ CILINDRO ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0, 1.1, 0));
        
        var ScalingMat = glm.scale(glm.vec3(1.2, 1.2, 0.45));
        
        var myModelMatrix = TranslationMat['*'](RotMat['*'](ScalingMat));
        
        this.cilindro.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.cilindro.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        //------------------------ CILINDRO ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0, 1.55, 0));
        
        var ScalingMat = glm.scale(glm.vec3(1.3, 1.3, 0.2));
        
        var myModelMatrix = TranslationMat['*'](RotMat['*'](ScalingMat));
        
        this.cilindro2.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.cilindro2.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);


    }

}


</script>




<h2>Car</h2>

<p>
    The car was created using cylinders for the wheels, a sphere to represent the driver, a cylinder for the chassis, and cubes for the spoiler.
</p>

In [26]:
%%html

<script>

class carro extends CGRAobject{
    constructor(glcontext,r,g,b, ncarro){
        super(glcontext);
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        var fragsrcL = document.getElementById("my-fragment-shaderL").text;
        var vertsrcL = document.getElementById("my-vertex-shaderL").text;
        
        this.shaderprogL = new DEECshader(this.gl);
        this.shaderprogL.srcShaders(vertsrcL,fragsrcL);
        
        if(ncarro == 1){
            this.texture = new CGRAtexture(this.gl);
            this.texture.load("Images/azul.jpg");
            
            this.textureJante = new CGRAtexture(this.gl);
            this.textureJante.load("Images/vermelho.jpg");
            
            this.texturePessoa = new CGRAtexture(this.gl);
            this.texturePessoa.load("Images/Pessoa1.png");
        }else{
            this.texture = new CGRAtexture(this.gl);
            this.texture.load("Images/vermelho.jpg");
            
            this.textureJante = new CGRAtexture(this.gl);
            this.textureJante.load("Images/azul.jpg");
            
            this.texturePessoa = new CGRAtexture(this.gl);
            this.texturePessoa.load("Images/Pessoa2.jpg");
        }
        
        this.texturePneu = new CGRAtexture(this.gl);
        this.texturePneu.load("Images/pneu.jpg");
        
        this.textureSpoiler = new CGRAtexture(this.gl);
        this.textureSpoiler.load("Images/poste.jpg");
        
        
        // ntriangulos, red, green, blue, samecolor
        //roda
        this.jante = new cilindroL(this.gl,30,b,g,r,1);
        this.jante.settexture(this.textureJante);
        this.jante.setShader(this.shaderprogL);
        this.pneu = new cilindroL(this.gl,30,0.5,0.5,0.5,1);
        this.pneu.settexture(this.texturePneu);
        this.pneu.setShader(this.shaderprogL);
        
        //roda1
        this.jante1 = new cilindroL(this.gl,30,b,g,r,1);
        this.jante1.settexture(this.textureJante);
        this.jante1.setShader(this.shaderprogL);
        this.pneu1 = new cilindroL(this.gl,30,0.5,0.5,0.5,1);
        this.pneu1.settexture(this.texturePneu);
        this.pneu1.setShader(this.shaderprogL);
        
        //roda2
        this.jante2 = new cilindroL(this.gl,30,b,g,r,1);
        this.jante2.settexture(this.textureJante);
        this.jante2.setShader(this.shaderprogL);
        this.pneu2 = new cilindroL(this.gl,30,0.5,0.5,0.5,1);
        this.pneu2.settexture(this.texturePneu);
        this.pneu2.setShader(this.shaderprogL);
        
        //roda3
        this.jante3 = new cilindroL(this.gl,30,b,g,r,1);
        this.jante3.settexture(this.textureJante);
        this.jante3.setShader(this.shaderprogL);
        this.pneu3 = new cilindroL(this.gl,30,0.5,0.5,0.5,1);
        this.pneu3.settexture(this.texturePneu);
        this.pneu3.setShader(this.shaderprogL);
        
        //chassis
        //this.chassis = new cilindro(this.gl,30,r,g,b,1);
        //this.chassis.setShader(this.shaderprog);
        
        this.chassis = new cilindroL(this.gl,30,r,g,b,1);
        this.chassis.settexture(this.texture);
        this.chassis.setShader(this.shaderprogL);
        
        //cabeça
        this.cabeca = new esferaL(this.gl,30,225/255,173/255,164/255,1.0);
        this.cabeca.settexture(this.texturePessoa);
        this.cabeca.setShader(this.shaderprogL);
        
        //spoiler
        this.spoiler = new cuboL(this.gl, 1, 1, 1, 1.0, 0);
        this.spoiler.settexture(this.textureSpoiler);
        this.spoiler.setShader(this.shaderprogL);
        //spoiler1
        this.spoiler1 = new cuboL(this.gl, b, g, r,1.0, 0);
        this.spoiler1.settexture(this.textureJante);
        this.spoiler1.setShader(this.shaderprogL);
        //spoiler
        this.spoiler2 = new cuboL(this.gl, b, g, r,1.0, 0);
        this.spoiler2.settexture(this.textureJante);
        this.spoiler2.setShader(this.shaderprogL);
        
        
    }
    
    drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2){
        //------------------------ jante ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0,0,0));
        
        var ScalingMat = glm.scale(glm.vec3(0.5, 0.5, 0.55));
        
        var RotMat = glm.rotate(glm.mat4(1.0),glm.radians(-90),glm.vec3(0,1,0));
        
        var myModelMatrix = RotMat['*'](TranslationMat['*'](ScalingMat));
        
        this.jante.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.jante.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        //------------------------ pneu ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0,0,0));
        
        var ScalingMat = glm.scale(glm.vec3(1.0, 1.0, 0.5));
        
        var RotMat = glm.rotate(glm.mat4(1.0),glm.radians(-90),glm.vec3(0,1,0));
        
        var myModelMatrix = RotMat['*'](TranslationMat['*'](ScalingMat));
        
        this.pneu.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.pneu.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        
        //------------------------ jante1 ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0,0,-4.05));
        
        var ScalingMat = glm.scale(glm.vec3(0.5, 0.5, 0.55));
        
        var RotMat = glm.rotate(glm.mat4(1.0),glm.radians(-90),glm.vec3(0,1,0));
        
        var myModelMatrix = RotMat['*'](TranslationMat['*'](ScalingMat));
        
        this.jante1.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.jante1.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        //------------------------ pneu1 ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0,0,-4.0));
        
        var ScalingMat = glm.scale(glm.vec3(1.0, 1.0, 0.5));
        
        var RotMat = glm.rotate(glm.mat4(1.0),glm.radians(-90),glm.vec3(0,1,0));
        
        var myModelMatrix = RotMat['*'](TranslationMat['*'](ScalingMat));
        
        this.pneu1.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.pneu1.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        //------------------------ jante2 ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0,6,-4.05));
        
        var ScalingMat = glm.scale(glm.vec3(0.5, 0.5, 0.55));
        
        var RotMat = glm.rotate(glm.mat4(1.0),glm.radians(-90),glm.vec3(0,1,0));
        
        var myModelMatrix = RotMat['*'](TranslationMat['*'](ScalingMat));
        
        this.jante2.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.jante2.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        //------------------------ pneu2 ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0,6,-4.0));
        
        var ScalingMat = glm.scale(glm.vec3(1.0, 1.0, 0.5));
        
        var RotMat = glm.rotate(glm.mat4(1.0),glm.radians(-90),glm.vec3(0,1,0));
        
        var myModelMatrix = RotMat['*'](TranslationMat['*'](ScalingMat));
        
        this.pneu2.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.pneu2.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
         //------------------------ jante3 ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0,6,-0.05));
        
        var ScalingMat = glm.scale(glm.vec3(0.5, 0.5, 0.55));
        
        var RotMat = glm.rotate(glm.mat4(1.0),glm.radians(-90),glm.vec3(0,1,0));
        
        var myModelMatrix = RotMat['*'](TranslationMat['*'](ScalingMat));
        
        this.jante3.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.jante3.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        //------------------------ pneu3 ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0,6,0));
        
        var ScalingMat = glm.scale(glm.vec3(1.0, 1.0, 0.5));
        
        var RotMat = glm.rotate(glm.mat4(1.0),glm.radians(-90),glm.vec3(0,1,0));
        
        var myModelMatrix = RotMat['*'](TranslationMat['*'](ScalingMat));
        
        this.pneu3.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.pneu3.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        //------------------------ chassis ------------------------
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(1.5,-0.5,0));
        
        //para cone var ScalingMat = glm.scale(glm.vec3(2,2,6));
        var ScalingMat = glm.scale(glm.vec3(2,1,3.5));
        
        var RotMat = glm.rotate(glm.mat4(1.0),glm.radians(-90),glm.vec3(1,0,0));
        
        var myModelMatrix = TranslationMat['*'](RotMat['*']((ScalingMat)));
        
        this.chassis.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.chassis.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);

        //------------------------ cabeça ------------------------
        
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(1.5,2,-1.5));
        
        var ScalingMat = glm.scale(glm.vec3(0.4,0.4,0.4));
        
        var RotMat = glm.rotate(glm.mat4(1.0),glm.radians(-90),glm.vec3(1,0,0));
        
        var myModelMatrix = TranslationMat['*'](RotMat['*']((ScalingMat)));
        
        this.cabeca.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.cabeca.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);  
        //------------------------ spoiler ------------------------
         var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(1.5,-1.2,-2));
        
        var ScalingMat = glm.scale(glm.vec3(5,0.6,0.8));
        
        var RotMat = glm.rotate(glm.mat4(1.0),glm.radians(-90),glm.vec3(1,0,0));
        
        var myModelMatrix = TranslationMat['*'](RotMat['*']((ScalingMat)));
        
        this.spoiler.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.spoiler.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2); 
        //------------------------ spoiler2 ------------------------
         var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0.5,-0.75,-1.5));
        
        var ScalingMat = glm.scale(glm.vec3(0.4,2,0.4));
        
        var RotMat = glm.rotate(glm.mat4(1.0),glm.radians(50),glm.vec3(1,0,0));
        
        var myModelMatrix = TranslationMat['*'](RotMat['*']((ScalingMat)));
        
        this.spoiler1.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.spoiler1.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);
        //------------------------ spoiler ------------------------
         var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(2.5,-0.75,-1.5));
        
        var ScalingMat = glm.scale(glm.vec3(0.4,2,0.4));
        
        var RotMat = glm.rotate(glm.mat4(1.0),glm.radians(50),glm.vec3(1,0,0));
        
        var myModelMatrix = TranslationMat['*'](RotMat['*']((ScalingMat)));
        
        this.spoiler2.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.spoiler2.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2); 
        
        
    }

}


</script>


<canvas id="myCanvasCarro" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp">

class myappCarro extends DEECapp{
    
    angRotacao = 0;
    
    initialize(){
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;

        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        
        this.gl.enable(this.gl.DEPTH_TEST);
        
        // Create cube object
        //Carro azul
        this.mycarro = new carro(this.gl,0.0,0.0,1.0,1);
        this.mycarro.setShader(this.shaderprog);

    }
    
    render(){
       
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        
        this.angRotacao = this.angRotacao + 3;
        
        
        // TRANSLAÇÃO
        var TranslationMat = glm.translate(glm.mat4(1.0), glm.vec3(12.0,10.0,10.0));
        
        // ROTAÇÃO
        var myRotAxis = glm.vec3(0.0,0.0,1.0);
        var RotMat = glm.rotate(glm.radians(this.angRotacao),myRotAxis);

        // var myModelMatrix = TranslationMat['*'](RotMat['*'](ScalingMat));
        var myModelMatrix = TranslationMat['*'](RotMat);
        //var myModelMatrix = TranslationMat;
        

        // TRANSFORMAÇÃO DE VISTA/PERSPETIVA
        var ViewMatrix = glm.lookAt(
            glm.vec3(-3,-3,-3),       // cameraPosition
            glm.vec3(8,8,8), // cameraTarget
            glm.vec3(1,1,0)        // upVector
        );
        
        // PROJEÇÃO - FoV, aspectratio, nearplane, farplane
        var ProjectionMatrix = glm.perspective(glm.radians(45.0), 1.0, 0.2, 100.0);
        
        // Computar a transformação aplicada a cada vértice do modelo
        var MVPmatrix = ProjectionMatrix['*'](ViewMatrix['*'](myModelMatrix));
        
        this.mycarro.setModelTransformation(myModelMatrix);
        
        // Utilizar o método drawit fornecido
        this.mycarro.drawit(ViewMatrix, ProjectionMatrix, 1, 0, 0);

        
    }
}

var appCarro = new myappCarro('myCanvasCarro');

appCarro.run();
</script>


<h2>Light Post</h2>

<p>
    The light post was created using a cone and two cubes for support.
</p>

In [27]:
%%html

<script>

class holofote extends CGRAobject{
    constructor(glcontext){
        super(glcontext);
        
        var fragsrcL = document.getElementById("my-fragment-shaderL").text;
        var vertsrcL = document.getElementById("my-vertex-shaderL").text;
        
        this.shaderprogL = new DEECshader(this.gl);
        this.shaderprogL.srcShaders(vertsrcL,fragsrcL);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("Images/silver.jpg");
        
        // ntriangulos, red, green, blue, samecolor
        this.cubo = new cuboL(this.gl, 0.75, 0.75, 0.75, 1,0);
        this.cubo.settexture(this.texture);
        this.cubo.setShader(this.shaderprogL);
                
        // ntriangulos, red, green, blue, samecolor
        this.cone = new coneL(this.gl,30,0.75, 0.75, 0.75, 1);
        this.cone.settexture(this.texture);
        this.cone.setShader(this.shaderprogL); 
        
        
    }
    
    drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2){
        
        //------------------------ POSTE 1 ------------------------
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(0, 0, 0));
        
        var RotMat = glm.rotate(glm.radians(0),glm.vec3(0.0,1.0,0.0));
        
        var ScalingMat = glm.scale(glm.vec3(0.5,10,0.5));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.cubo.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.cubo.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2); 
        
        //------------------------ POSTE 2 ------------------------
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(1.35, 4, 0));

        var RotMat = glm.rotate(glm.radians(0),glm.vec3(0.0,1.0,0.0));
        
        var ScalingMat = glm.scale(glm.vec3(4,0.51,0.51));
        
        var myModelMatrix = TranslationMat['*'](ScalingMat);
        
        this.cubo.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.cubo.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2); 
        
    
        //------------------------ CONE ------------------------
        var TranslationMat = glm.translate(glm.mat4(1.0),glm.vec3(2.75, 3, 0));
        
        var RotMat = glm.rotate(glm.radians(-90),glm.vec3(1.0,0.0,0.0));
        
        var ScalingMat = glm.scale(glm.vec3(1, 1, 0.6));
        
        var myModelMatrix = TranslationMat['*'](RotMat['*'](ScalingMat));
        
        this.cone.setModelTransformation(this.modelMat['*'](myModelMatrix));
        this.cone.drawit(ViewMatrix,ProjectionMatrix, NoiteDia, lamp1, lamp2, lightL1, lightL2);

    }

}


</script>


<canvas id="myCanvasholofote" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp">

class myappholofote extends DEECapp{
    
    angRotacao = 0;
    
    initialize(){
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;

        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        
        this.gl.enable(this.gl.DEPTH_TEST);
        
        // Create cube object
        this.myholofote = new holofote(this.gl);
        this.myholofote.setShader(this.shaderprog);

    }
    
    render(){
       
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        
        this.angRotacao = this.angRotacao + 3;
        
        
        // TRANSLAÇÃO
        var TranslationMat = glm.translate(glm.mat4(1.0), glm.vec3(0, 0, 0));
        
        // ROTAÇÃO
        var RotMat = glm.rotate(glm.radians(this.angRotacao),glm.vec3(0.0,1.0,0.0));

        // var myModelMatrix = TranslationMat['*'](RotMat['*'](ScalingMat));
        var myModelMatrix = TranslationMat['*'](RotMat);
        

        // TRANSFORMAÇÃO DE VISTA/PERSPETIVA
        var ViewMatrix = glm.lookAt(
            glm.vec3(7,7,7),       // cameraPosition
            glm.vec3(0, 0, 0), // cameraTarget
            glm.vec3(0,1,0)        // upVector
        );
        
        // PROJEÇÃO - FoV, aspectratio, nearplane, farplane
        var ProjectionMatrix = glm.perspective(glm.radians(45.0), 1.0, 0.1, 100.0);
        
        this.myholofote.setModelTransformation(myModelMatrix);
        
        // Utilizar o método drawit fornecido
        this.myholofote.drawit(ViewMatrix, ProjectionMatrix, 1, 0, 0);

        
    }
}

var appholofote = new myappholofote('myCanvasholofote');

appholofote.run();
</script>

<h1>Scenario</h1>

<p>
    For the scenario, all previously created objects were added, such as two cars, four benches, some trees, among other objects. The track was created using two cylinders.
</p>

<p>
    For the movement of the cars, a transformation was performed, consisting of a translation to the origin + rotation + translation from the origin. The rotation is around the Y axis, so when performing the rotation and translating from the origin to the next position, these are done according to the angles <strong>angRotacao1</strong> and <strong>angRotacao2</strong> for cars 1 and 2, respectively.
</p>

<p>
    When the "c" key is pressed to view from the car's perspective, it "follows" the same transformation applied to the car.
</p>

<p>
    To manipulate the viewpoint, day/night scenario, and lights, the following keys are used:
</p>

<ul>
    <li><strong>a</strong> - rotate left</li>
    <li><strong>d</strong> - rotate right</li>
    <li><strong>s</strong> - move backward</li>
    <li><strong>w</strong> - move forward</li>
    <li><strong>p</strong> - top view (increase Y)</li>
    <li><strong>l</strong> - bottom view (decrease Y)</li>
    <li><strong>c</strong> - key to switch viewpoint</li>
    <li><strong>n</strong> - key to change day/night</li>
    <li><strong>1</strong> - Light 1 (white + RGB)</li>
    <li><strong>2</strong> - Light 2 (white + RGB)</li>
</ul>

<p>
    The viewpoint is manipulated through several variables:
</p>

<p>
    When rotating left or right, the angle <strong>rotateAngle</strong> is increased or decreased, respectively.
</p>

<p>
    When moving forward or backward, <strong>positionX</strong> and <strong>positionY</strong> are increased or decreased, respectively. These variables are calculated using the formulas:
</p>

<pre>
positionX +/-= 0.5 * cos(rotateAngle); 
positionZ +/-= 0.5 * sin(rotateAngle); 
</pre>

<p>
    To achieve a higher or lower view, the variable <strong>up</strong> is increased or decreased, respectively.
</p>

<p>
    The previous variables are used to define <strong>cameraPosition</strong> and <strong>cameraTarget</strong> for the viewpoint as follows:
</p>

<pre>
cameraPosition = (positionX, 0.2 + up, positionZ);
cameraTarget = (positionX + cos(rotateAngle), 0.2 + up, positionZ + sin(rotateAngle));
</pre>

In [28]:

%%html
<pre id="matrix"></pre>

<canvas id="myCanvasScenario" width="800" height="800" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp">

class myappScenario extends DEECapp{
    
    num = 0;
    
    // Variáveis para o movimento do carro à volta da pista
    
    angRotacao1 = 0;
    angRotacao2 = 130;
    positionXCar = 0; 
    positionYCar = 0; 
    positionZCar = 0; 
    
    cameraPositionCarro1 = glm.vec3(0,0,0);
    cameraTargetCarro1 = glm.vec3(0,0,0);
    
    // Variável para teclas premidas
    key = 0;
    
    // Variável para mudar a vista
    mudarVista=0;
    
    // Variável para mudar dia/noite
    mudarNoiteDia = 1;
    NoiteDia = 1;
    
    lamp1 = 0;
    lamp2 = 0;
    
    counterLamp1 = 0;
    counterLamp2 = 0;
    
    // Variáveis para a ViewMatrix
    
    cameraPosition = glm.vec3(0,0,0);
    cameraTarget = glm.vec3(0,0,0);
    
    positionX = -62;
    positionZ = 18;
    
    rotateAngle = -15;
    rotateAngleCarro1 = -15;
    
    forward = 0.5;
    
    up = 10;
    
    initialize(){
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        var fragsrcT = document.getElementById("my-fragment-shaderT").text;
        var vertsrcT = document.getElementById("my-vertex-shaderT").text;
        
        var fragsrcL = document.getElementById("my-fragment-shaderL").text;
        var vertsrcL = document.getElementById("my-vertex-shaderL").text;
        

        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        this.shaderprogT = new DEECshader(this.gl);
        this.shaderprogT.srcShaders(vertsrcT,fragsrcT);
        
        this.shaderprogL = new DEECshader(this.gl);
        this.shaderprogL.srcShaders(vertsrcL,fragsrcL);
        
        
        this.gl.clearColor(0.67,0.923,0.99,1.0);
        
        this.gl.enable(this.gl.DEPTH_TEST);
        
        // Chão
        this.textureChao = new CGRAtexture(this.gl);
        this.textureChao.load("Images/sand.jpg");

        //r, g, b, samecolor, screen
        this.chao = new cuboL(this.gl,244/255,164/255,96/255,1.0, 1)
        this.chao.settexture(this.textureChao);
        this.chao.setShader(this.shaderprogL);
        
        // Pista1
        this.texturePista = new CGRAtexture(this.gl);
        this.texturePista.load("Images/pista.jpg");
        
        this.pista1 = new cilindroL(this.gl, 30, 0.5, 0.5, 0.5, 1);
        this.pista1.settexture(this.texturePista);
        this.pista1.setShader(this.shaderprogL);
        
        // Pista2
        this.textureRelva = new CGRAtexture(this.gl);
        this.textureRelva.load("Images/relva.jpg");

        this.pista2 = new cilindroL(this.gl, 30, 0,154/255,23/255, 1);
        this.pista2.settexture(this.textureRelva);
        this.pista2.setShader(this.shaderprogL);
        
        // Bancada1
        this.bancada1 = new bancada(this.gl,0.9,0,0, 2);
        this.bancada1.setShader(this.shaderprog);
        
        // Bancada2
        this.bancada2 = new bancada(this.gl,0,0,0.9, 1);
        this.bancada2.setShader(this.shaderprog);
        
        // Bancada3
        this.bancada3 = new bancada(this.gl,0.9,0,0, 2);
        this.bancada3.setShader(this.shaderprog);
        
        // Bancada4
        this.bancada4 = new bancada(this.gl,0,0,0.9, 1);
        this.bancada4.setShader(this.shaderprog);
        
        // Pinheiro1
        this.pinheiro1 = new pinheiro(this.gl);
        this.pinheiro1.setShader(this.shaderprog);
        
        // Pinheiro2
        this.pinheiro2 = new pinheiro(this.gl);
        this.pinheiro2.setShader(this.shaderprog);
        
        // Pinheiro3
        this.pinheiro3 = new pinheiro(this.gl);
        this.pinheiro3.setShader(this.shaderprog);
        
        // Arvore1
        this.arvore1 = new arvore(this.gl);
        this.arvore1.setShader(this.shaderprog);
        
        // Arvore2
        this.arvore2 = new arvore(this.gl);
        this.arvore2.setShader(this.shaderprog);
        
        // Arvore3
        this.arvore3 = new arvore(this.gl);
        this.arvore3.setShader(this.shaderprog);
        
        // Arvore4
        this.arvore4 = new arvore(this.gl);
        this.arvore4.setShader(this.shaderprog);
        
        
        // Arbusto1
        this.arbusto1 = new arbusto(this.gl,33/255,98/255,66/255);
        this.arbusto1.setShader(this.shaderprog);
        
        // Nuvem1
        this.nuvem1 = new nuvem(this.gl);
        this.nuvem1.setShader(this.shaderprog);
        
        // Nuvem2
        this.nuvem2 = new nuvem(this.gl);
        this.nuvem2.setShader(this.shaderprog);
        
        // Nuvem3
        this.nuvem3 = new nuvem(this.gl);
        this.nuvem3.setShader(this.shaderprog);
        
        // Sol
        this.textureSol = new CGRAtexture(this.gl);
        this.textureSol.load("Images/sun2.jpg");
        
        this.sol = new esferaL(this.gl,12,239/255,142/255,56/255, 1.0);
        this.sol.settexture(this.textureSol);
        this.sol.setShader(this.shaderprogT);
        
        // LinhaPartida
        this.texturePartida = new CGRAtexture(this.gl);
        this.texturePartida.load("Images/sol.jpg");
        
        this.linha = new cuboL(this.gl,0.9,0.9,0.9,1.0, 0)
        this.linha.settexture(this.texturePartida);
        this.linha.setShader(this.shaderprogL);
        
        // Poste1
        this.texturePoste = new CGRAtexture(this.gl);
        this.texturePoste.load("Images/silver.jpg");
        
        this.poste1 = new cuboL(this.gl,0.75,0.75,0.75,1.0, 0)
        this.poste1.settexture(this.texturePoste);
        this.poste1.setShader(this.shaderprogL);
        
        // Poste2
        this.poste2 = new cuboL(this.gl,0.75,0.75,0.75,1.0, 0)
        this.poste2.settexture(this.texturePoste)
        this.poste2.setShader(this.shaderprogL);
        
        // Suporte 
        this.textureSuporte = new CGRAtexture(this.gl);
        this.textureSuporte.load("Images/black.jpg");
        this.Suporte = new cuboL(this.gl,0.75,0.75,0.75,1.0, 0)
        this.Suporte.settexture(this.textureSuporte)
        this.Suporte.setShader(this.shaderprogL);
        
        // Taça
        this.taca = new taca(this.gl);
        this.taca.setShader(this.shaderprog);
        
        
        // Casa1
        this.casa1 = new casa(this.gl,64/225,224/225,208/255,65/255,85/255,90/255);
        this.casa1.setShader(this.shaderprog);
        
        // Holofote 
        this.holofote = new holofote(this.gl);
        this.holofote.setShader(this.shaderprog);
         
        
        //Carro1 azul
        this.carro1 = new carro(this.gl,0.0,0.0,1.0, 1);
        this.carro1.setShader(this.shaderprog);
        
        //Carro2 vermelho
        this.carro2 = new carro(this.gl,1.0,0.0,0.0, 2);
        this.carro2.setShader(this.shaderprog);
        
        // Create a texture to render to
        this.targetTextureWidth = 256;
        this.targetTextureHeight = 256;
        this.targetTexture = this.gl.createTexture();
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.targetTexture);
        
        this.gl.texImage2D(this.gl.TEXTURE_2D, 0, this.gl.RGBA,
                  this.targetTextureWidth, this.targetTextureHeight, 0,
                  this.gl.RGBA, this.gl.UNSIGNED_BYTE, null);

        // set the filtering so we don't need mips
        this.gl.texParameteri(this.gl.TEXTURE_2D, this.gl.TEXTURE_MIN_FILTER, this.gl.LINEAR);
        this.gl.texParameteri(this.gl.TEXTURE_2D, this.gl.TEXTURE_WRAP_S, this.gl.CLAMP_TO_EDGE);
        this.gl.texParameteri(this.gl.TEXTURE_2D, this.gl.TEXTURE_WRAP_T, this.gl.CLAMP_TO_EDGE);
        

      // Create and bind the framebuffer
      this.fb = this.gl.createFramebuffer();
      this.gl.bindFramebuffer(this.gl.FRAMEBUFFER, this.fb);

      this.gl.framebufferTexture2D(this.gl.FRAMEBUFFER, this.gl.COLOR_ATTACHMENT0, 
                                   this.gl.TEXTURE_2D, this.targetTexture, 0);

      // create a depth renderbuffer
      this.depthBuffer = this.gl.createRenderbuffer();
      this.gl.bindRenderbuffer(this.gl.RENDERBUFFER, this.depthBuffer);

      // make a depth buffer and the same size as the targetTexture
      this.gl.renderbufferStorage(this.gl.RENDERBUFFER, this.gl.DEPTH_COMPONENT16, 
                                  this.targetTextureWidth, this.targetTextureHeight);
      this.gl.framebufferRenderbuffer(this.gl.FRAMEBUFFER, this.gl.DEPTH_ATTACHMENT, 
                                      this.gl.RENDERBUFFER, this.depthBuffer);


        // Screen
        this.screen2 = new cuboL(this.gl, 0.25,0.25,0.25,1.0, 1);
        this.screen2.textureid = this.targetTexture;
        this.screen2.setShader(this.shaderprogT);
        
        
        // Posição inicial
        
        // inicializações da camara
        this.ProjectionMatrix = glm.perspective(glm.radians(80), 1.0,1.0, 100);
        this.ViewMatrix = glm.lookAt(glm.vec3(0,0,0),glm.vec3(0,0,-1),glm.vec3(0,1,0));
        this.front=0.5;
        
        
        window.addEventListener("keypress", function(evt){appScenario.keyprocess(evt);}, false);

    }
    
    keyprocess(evt){
        console.log('Key pressed. Keycode:'+evt.keyCode + ' Char: \'' + String.fromCharCode(evt.charCode)+'\'');
        if(evt.keyCode == 27){
            alert("You pressed the \"Escape\" key.");
        }
        console.log('key='+ this.key)
        this.key = evt.keyCode;
    }
    
    //---------------------------------------------------
    
    // the scene drawing was moved to a separate method, so it can be called 
    // as needed from the render method.
    drawscene(view, proj, num){
       
        //this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        
        // Teclas usadas, ascii
        // a = 97  - Rodar esquerda
        // s = 115 - Andar para Trás
        // w = 119 - Andar para frente
        // d = 100 - Rodar direita
        // c = 99  - Trocar vista
        // p = 112 - Vista de cima
        // l = 108 - Vista de baixo
        // n = 110 - Mudar dia/noite
        // 1 = 49 - Lâmpada 1
        // 2 = 50 - Lâmpada 2
        
        console.log("Tecla premida =",this.key);
        
        switch (this.key) {
            case 97:            // "a" - Rodar esquerda
                console.log('app key='+ this.key);
            
                this.rotateAngle -= 2;
                this.key=0;
            
                break;
            
            case 100:            // "d" - Rodar direita
                console.log('app key='+ this.key);
            
                this.rotateAngle += 2;
                this.key=0;
            
                break;
            
            case 119:            // "w" - Andar para frente
                console.log('app key='+ this.key);
            
                this.positionX += this.forward * Math.cos(glm.radians(this.rotateAngle)); 
                this.positionZ += this.forward * Math.sin(glm.radians(this.rotateAngle)); 
                

                this.key=0;
            
                break;
            
            case 115:            // "s" - Andar para trás
                console.log('app key='+ this.key);
    
                this.positionX -= this.forward * Math.cos(glm.radians(this.rotateAngle)); 
                this.positionZ -= this.forward * Math.sin(glm.radians(this.rotateAngle)); 
                
                this.key=0;
            
                break;
            
            case 112:            // "p" - Vista de cima 
                console.log('app key='+ this.key);
            
                this.up += 0.5;

                this.key=0;
            
                break;
            
            case 108:            // "l" - Vista de baixo
                console.log('app key='+ this.key);
            
                this.up -= 0.5;
        
                this.key=0;
            
                break;
            case 111:           //tecla "o" para ver a nossa posiçao
                console.log(this.cameraPosition.array);
                console.log(this.cameraTarget.array);
                this.key=0;
                break;
            case 99:           //tecla "c" mudar a vista para o carro
                if(this.mudarVista == 0){
                    this.mudarVista = 1;
                    }
                else{
                    this.mudarVista = 0;
                    }
                this.key=0;
                break;
            case 110:          //tecla "n" mudar dia/noite
            
                if(this.mudarNoiteDia == 0){ //Noite
                    this.mudarNoiteDia = 1; 
                    this.NoiteDia = 0;   
                    this.textureSol.load("Images/lua.jpg");
                }
                else{ // Dia
                    this.mudarNoiteDia = 0;
                    this.NoiteDia = 1; 
                     
                    this.textureSol.load("Images/sun2.jpg");
                }
                this.key=0;
                break;
            
            case 49:         //tecla "1" 
                
                    if(this.NoiteDia == 0){
                        this.lamp1 = 1;
                        this.counterLamp1++;
                        
                        if(this.counterLamp1 == 1){
                            this.lightL1 = glm.vec3(0.5, 0.5, 0.5);
                        }
                        if(this.counterLamp1 == 2){
                            this.lightL1 = glm.vec3(1.0, 0.0, 0.0);
                        }
                        if(this.counterLamp1 == 3){
                            this.lightL1 = glm.vec3(0.0, 1.0, 0.0);
                        }
                        if(this.counterLamp1 == 4){
                            this.lightL1 = glm.vec3(0.0, 0.0, 1.0);
                        }
                        
                        if(this.counterLamp1 > 4){
                            this.lamp1 = 0;
                            this.counterLamp1 = 0;
                        }
                    }

                this.key=0;
                break;
            
            case 50:         //tecla "2" 
                    if(this.NoiteDia == 0){
                        this.lamp2 = 1;
                        this.counterLamp2++;
                        
                        if(this.counterLamp2 == 1){
                            this.lightL2 = glm.vec3(0.5, 0.5, 0.5);
                        }
                        if(this.counterLamp2 == 2){
                            this.lightL2 = glm.vec3(1.0, 0.0, 0.0);
                        }
                        if(this.counterLamp2 == 3){
                            this.lightL2 = glm.vec3(0.0, 1.0, 0.0);
                        }
                        if(this.counterLamp2 == 4){
                            this.lightL2 = glm.vec3(0.0, 0.0, 1.0);
                        }
                        
                        if(this.counterLamp2 > 4){
                            this.lamp2 = 0;
                            this.counterLamp2 = 0;
                        }
                    }

                this.key=0;
                break;
            
            default:
                this.key=0;
                break;
        }
        
        if(this.mudarVista == 0){
            this.cameraPosition = glm.vec3(this.positionX, 0.2 + this.up , this.positionZ);
            this.cameraTarget = glm.vec3(this.positionX + Math.cos(glm.radians(this.rotateAngle)),
                                         0.2 + this.up,
                                         this.positionZ + Math.sin(glm.radians(this.rotateAngle)));


            // TRANSFORMAÇÃO DE VISTA/PERSPETIVA
            var ViewMatrix = glm.lookAt(
                this.cameraPosition,       // cameraPosition
                this.cameraTarget,         // cameraTarget
                glm.vec3(0,1,0)            // upVector
            );
        }
        
        var ProjectionMatrix = glm.perspective(glm.radians(60.0), 1.0, 1.0, 150.0);
        
        if(this.mudarVista == 1)
        {

            //this.cameraPosition = glm.vec3(myModelMatrixCarro1[3][0],myModelMatrixCarro1[3][1],myModelMatrixCarro1[3][2]);
            
            //this.cameraTargetCarro1 = glm.vec3(-5,2,0);
            
            console.log("sa=" + this.rotateAngleCarro1);
            var ViewMatrix = glm.lookAt(
                this.cameraPositionCarro1,
                this.cameraTargetCarro1,
                glm.vec3(0,1,0) 
            );
    
        }

        if(num == 1){
            var ViewMatrix = glm.lookAt(
                this.cameraPositionCarro1,
                this.cameraTargetCarro1,
                glm.vec3(0,1,0) 
            );
            
        }
    
        
        //------------------------------ CHÃO  ------------------------------
        var TranslationChao = glm.translate(glm.mat4(1.0),glm.vec3(0, -2, 0));
        
        var ScalingChao = glm.scale(glm.vec3(150, 5, 150));
        
        var myModelMatrixChao = TranslationChao['*'](ScalingChao);
        
        this.chao.setModelTransformation(myModelMatrixChao);
        this.chao.drawit(ViewMatrix,ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2,this.lightL1,this.lightL2);
        
        //------------------------------ PISTA 1  ------------------------------
        var TranslationPista1 = glm.translate(glm.mat4(1.0),glm.vec3(0,0,-0.1));
        
        var ScalingPista1 = glm.scale(glm.vec3(30,30, 0.1));
        
        var RotMatPista1 = glm.rotate(glm.mat4(1.0),glm.radians(90),glm.vec3(1,0,0));
        
        var myModelMatrixPista1 = RotMatPista1['*'](TranslationPista1['*'](ScalingPista1));
        
        this.pista1.setModelTransformation(myModelMatrixPista1);
        this.pista1.drawit(ViewMatrix,ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2, this.lightL1,this.lightL2);
        
        //------------------------------ PISTA 2  ------------------------------
        var TranslationPista2 = glm.translate(glm.mat4(1.0),glm.vec3(0,0,-0.15));
        
        var ScalingPista2 = glm.scale(glm.vec3(15,15, 0.15));
        
        var RotMatPista2 = glm.rotate(glm.mat4(1.0),glm.radians(90),glm.vec3(1,0,0));
        
        var myModelMatrixPista2 = RotMatPista2['*'](TranslationPista2['*'](ScalingPista2));
        
        this.pista2.setModelTransformation(myModelMatrixPista2);
        this.pista2.drawit(ViewMatrix,ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2, this.lightL1,this.lightL2);
        
        //------------------------------ BANCADA 1  ------------------------------
        var TranslationBancada1 = glm.translate(glm.mat4(1.0),glm.vec3(-35,0,12));
        
        var ScalingBancada1 = glm.scale(glm.vec3(1,1, 1));
        
        var RotMatBancada1 = glm.rotate(glm.mat4(1.0),glm.radians(180),glm.vec3(0,1,0));
        
        var myModelMatrixBancada1 = RotMatBancada1['*'](TranslationBancada1['*'](ScalingBancada1));
        
        this.bancada1.setModelTransformation(myModelMatrixBancada1);
        this.bancada1.drawit(ViewMatrix,ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2, this.lightL1,this.lightL2);
        
        //------------------------------ BANCADA 2  ------------------------------
        var TranslationBancada2 = glm.translate(glm.mat4(1.0),glm.vec3(-35,0,-12));
        
        var ScalingBancada2 = glm.scale(glm.vec3(1,1, 1));
        
        var RotMatBancada2 = glm.rotate(glm.mat4(1.0),glm.radians(180),glm.vec3(0,1,0));
        
        var myModelMatrixBancada2 = RotMatBancada2['*'](TranslationBancada2['*'](ScalingBancada2));
        
        this.bancada2.setModelTransformation(myModelMatrixBancada2);
        this.bancada2.drawit(ViewMatrix,ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2, this.lightL1,this.lightL2);
        
        //------------------------------ BANCADA 3  ------------------------------
        var TranslationBancada3 = glm.translate(glm.mat4(1.0),glm.vec3(-35,0,10));
        
        var ScalingBancada3 = glm.scale(glm.vec3(1,1, 1));
        
        var RotMatBancada3 = glm.rotate(glm.mat4(1.0),glm.radians(225),glm.vec3(0,1,0));
        //RotMatBancada3 = glm.rotate(glm.mat4(1.0),glm.radians(-225),glm.vec3(0,1,0));
        
        var myModelMatrixBancada3 = RotMatBancada3['*'](TranslationBancada3['*'](ScalingBancada3));
        
        this.bancada3.setModelTransformation(myModelMatrixBancada3);
        this.bancada3.drawit(ViewMatrix,ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2, this.lightL1,this.lightL2);
        
        //------------------------------ BANCADA 4  ------------------------------
        var TranslationBancada4 = glm.translate(glm.mat4(1.0),glm.vec3(-35,0,-10));
        
        var ScalingBancada4 = glm.scale(glm.vec3(1,1, 1));
        
        var RotMatBancada4 = glm.rotate(glm.mat4(1.0),glm.radians(135),glm.vec3(0,1,0));
        //RotMatBancada3 = glm.rotate(glm.mat4(1.0),glm.radians(-225),glm.vec3(0,1,0));
        
        var myModelMatrixBancada4 = RotMatBancada4['*'](TranslationBancada4['*'](ScalingBancada4));
        
        this.bancada4.setModelTransformation(myModelMatrixBancada4);
        this.bancada4.drawit(ViewMatrix,ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2, this.lightL1,this.lightL2);
        
        //------------------------------ PINHEIRO 1  ------------------------------
        var TranslationPinheiro1 = glm.translate(glm.mat4(1.0),glm.vec3(35,0,-3));
        
        var ScalingPinheiro1 = glm.scale(glm.vec3(1,1, 1));
        
        var RotMatPinheiro1 = glm.rotate(glm.mat4(1.0),glm.radians(90),glm.vec3(1,0,0));
        
        var myModelMatrixPinheiro1 = RotMatPinheiro1['*'](TranslationPinheiro1['*'](ScalingPinheiro1));
        
        this.pinheiro1.setModelTransformation(myModelMatrixPinheiro1);
        this.pinheiro1.drawit(ViewMatrix,ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2, this.lightL1,this.lightL2);
        
        //------------------------------ PINHEIRO 2  ------------------------------
        var TranslationPinheiro2 = glm.translate(glm.mat4(1.0),glm.vec3(35,25,-3));
        
        var ScalingPinheiro2 = glm.scale(glm.vec3(1,1, 1));
        
        var RotMatPinheiro2 = glm.rotate(glm.mat4(1.0),glm.radians(90),glm.vec3(1,0,0));
        
        var myModelMatrixPinheiro2 = RotMatPinheiro1['*'](TranslationPinheiro2['*'](ScalingPinheiro2));
        
        this.pinheiro2.setModelTransformation(myModelMatrixPinheiro2);
        this.pinheiro2.drawit(ViewMatrix,ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2, this.lightL1,this.lightL2);
        
        //------------------------------ PINHEIRO 3  ------------------------------
        var TranslationPinheiro3 = glm.translate(glm.mat4(1.0),glm.vec3(35,-25,-3));
        
        var ScalingPinheiro3 = glm.scale(glm.vec3(1,1, 1));
        
        var RotMatPinheiro3 = glm.rotate(glm.mat4(1.0),glm.radians(90),glm.vec3(1,0,0));
        
        var myModelMatrixPinheiro3 = RotMatPinheiro3['*'](TranslationPinheiro3['*'](ScalingPinheiro3));
        
        this.pinheiro3.setModelTransformation(myModelMatrixPinheiro3);
        this.pinheiro3.drawit(ViewMatrix,ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2, this.lightL1,this.lightL2);
        
        //------------------------------ ARVORE 1  ------------------------------
        var TranslationArvore1 = glm.translate(glm.mat4(1.0),glm.vec3(-27,-22,-2.5));
        
        var ScalingArvore1 = glm.scale(glm.vec3(0.7,0.7, 0.7));
        
        var RotMatArvore1 = glm.rotate(glm.mat4(1.0),glm.radians(90),glm.vec3(1,0,0));
        
        var myModelMatrixArvore1 = RotMatArvore1['*'](TranslationArvore1['*'](ScalingArvore1));
        
        this.arvore1.setModelTransformation(myModelMatrixArvore1);
        this.arvore1.drawit(ViewMatrix,ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2, this.lightL1,this.lightL2);
        
        //------------------------------ ARVORE 2  ------------------------------
        var TranslationArvore2 = glm.translate(glm.mat4(1.0),glm.vec3(-27,22,-2.5));
        
        var ScalingArvore2 = glm.scale(glm.vec3(0.7,0.7, 0.7));
        
        var RotMatArvore2 = glm.rotate(glm.mat4(1.0),glm.radians(90),glm.vec3(1,0,0));
        
        var myModelMatrixArvore2 = RotMatArvore2['*'](TranslationArvore2['*'](ScalingArvore2));
        
        this.arvore2.setModelTransformation(myModelMatrixArvore2);
        this.arvore2.drawit(ViewMatrix,ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2, this.lightL1,this.lightL2);
        
         //------------------------------ ARVORE 3  ------------------------------
        var TranslationArvore3 = glm.translate(glm.mat4(1.0),glm.vec3(35,-35,-14));
        
        var ScalingArvore3 = glm.scale(glm.vec3(3,3, 3));
        
        var RotMatArvore3 = glm.rotate(glm.mat4(1.0),glm.radians(90),glm.vec3(1,0,0));
        
        var myModelMatrixArvore3 = RotMatArvore3['*'](TranslationArvore3['*'](ScalingArvore3));
        
        this.arvore3.setModelTransformation(myModelMatrixArvore3);
        this.arvore3.drawit(ViewMatrix,ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2, this.lightL1,this.lightL2);
        
        //------------------------------ ARVORE 4  ------------------------------
        var TranslationArvore4 = glm.translate(glm.mat4(1.0),glm.vec3(35,35,-14));
        
        var ScalingArvore4 = glm.scale(glm.vec3(3,3, 3));
        
        var RotMatArvore4 = glm.rotate(glm.mat4(1.0),glm.radians(90),glm.vec3(1,0,0));
        
        var myModelMatrixArvore4 = RotMatArvore4['*'](TranslationArvore4['*'](ScalingArvore4));
        
        this.arvore4.setModelTransformation(myModelMatrixArvore4);
        this.arvore4.drawit(ViewMatrix,ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2, this.lightL1,this.lightL2);
        
        //------------------------------ ARBUSTO 1  ------------------------------
        var TranslationArbusto1 = glm.translate(glm.mat4(1.0),glm.vec3(0,1,-33));
        
        var ScalingArbusto1 = glm.scale(glm.vec3(0.7,0.7, 0.7));
        
        var RotMatArbusto1 = glm.rotate(glm.mat4(1.0),glm.radians(0),glm.vec3(1,0,0));
        
        var myModelMatrixArbusto1 = RotMatArbusto1['*'](TranslationArbusto1['*'](ScalingArbusto1));
        
        this.arbusto1.setModelTransformation(myModelMatrixArbusto1);
        this.arbusto1.drawit(ViewMatrix,ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2, this.lightL1,this.lightL2);
        
        //------------------------------ NUVEM 1  ------------------------------
        var TranslationNuvem1 = glm.translate(glm.mat4(1.0),glm.vec3(0,20,20));
        
        var ScalingNuvem1 = glm.scale(glm.vec3(0.7,0.7, 0.7));
        
        var RotMatNuvem1 = glm.rotate(glm.mat4(1.0),glm.radians(90),glm.vec3(0,1,0));
        
        var myModelMatrixNuvem1 = RotMatNuvem1['*'](TranslationNuvem1['*'](ScalingNuvem1));
        
        this.nuvem1.setModelTransformation(myModelMatrixNuvem1);
        this.nuvem1.drawit(ViewMatrix,ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2, this.lightL1,this.lightL2);
        
        //------------------------------ NUVEM 2  ------------------------------
        var TranslationNuvem2 = glm.translate(glm.mat4(1.0),glm.vec3(15,25,20));
        
        var ScalingNuvem2 = glm.scale(glm.vec3(0.7,0.7, 0.7));
        
        var RotMatNuvem2 = glm.rotate(glm.mat4(1.0),glm.radians(90),glm.vec3(0,1,0));
        
        var myModelMatrixNuvem2 = RotMatNuvem2['*'](TranslationNuvem2['*'](ScalingNuvem2));
        
        this.nuvem2.setModelTransformation(myModelMatrixNuvem2);
        this.nuvem2.drawit(ViewMatrix,ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2, this.lightL1,this.lightL2);
        
        //------------------------------ NUVEM 3  ------------------------------
        var TranslationNuvem3 = glm.translate(glm.mat4(1.0),glm.vec3(-15,25,20));
        
        var ScalingNuvem3 = glm.scale(glm.vec3(0.7,0.7, 0.7));
        
        var RotMatNuvem3 = glm.rotate(glm.mat4(1.0),glm.radians(90),glm.vec3(0,1,0));
        
        var myModelMatrixNuvem3 = RotMatNuvem3['*'](TranslationNuvem3['*'](ScalingNuvem3));
        
        this.nuvem3.setModelTransformation(myModelMatrixNuvem3);
        this.nuvem3.drawit(ViewMatrix,ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2, this.lightL1,this.lightL2);
        
        //------------------------------ SOL   ------------------------------
        //var TranslationSol = glm.translate(glm.mat4(1.0),glm.vec3(25,-5,70));
        var TranslationSol = glm.translate(glm.mat4(1.0),glm.vec3(40,0,0));
        
        var ScalingSol = glm.scale(glm.vec3(2.3,2.5, 2.5));
        
        var RotMatSol = glm.rotate(glm.mat4(1.0),glm.radians(90),glm.vec3(0,1,0));
        var RotMatSol2 = glm.rotate(glm.mat4(1.0),glm.radians(90),glm.vec3(1,0,0));
        
        var myModelMatrixSol = RotMatSol2['*'](RotMatSol['*'](TranslationSol['*'](ScalingSol)));
        
        this.sol.setModelTransformation(myModelMatrixSol);
        this.sol.drawit(ViewMatrix, ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2, this.lightL1,this.lightL2);
        
        //------------------------------ LINHA  ------------------------------
        var TranslationLinha = glm.translate(glm.mat4(1.0),glm.vec3(23, 4.2, 0));
        
        var ScalingLinha = glm.scale(glm.vec3(25, 0.8, 1));
        
        var myModelMatrixLinha = TranslationLinha['*'](ScalingLinha);
        
        this.linha.setModelTransformation(myModelMatrixLinha);
        this.linha.drawit(ViewMatrix,ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2, this.lightL1,this.lightL2);
        //------------------------------ POSTE 1  ------------------------------
        var TranslationPoste1 = glm.translate(glm.mat4(1.0),glm.vec3(31, 0, 0));
        
        var ScalingPoste1 = glm.scale(glm.vec3(1, 10, 1));
        
        var myModelMatrixPoste1 = TranslationPoste1['*'](ScalingPoste1);
        
        this.poste1.setModelTransformation(myModelMatrixPoste1);
        this.poste1.drawit(ViewMatrix,ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2, this.lightL1,this.lightL2);
        
        //------------------------------ POSTE 2  ------------------------------
        var TranslationPoste2 = glm.translate(glm.mat4(1.0),glm.vec3(14, 0, 0));
        
        var ScalingPoste2 = glm.scale(glm.vec3(1, 10, 1));
        
        var myModelMatrixPoste2 = TranslationPoste2['*'](ScalingPoste2);
        
        this.poste2.setModelTransformation(myModelMatrixPoste2);
        this.poste2.drawit(ViewMatrix,ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2, this.lightL1,this.lightL2);
        
         //------------------------------ CASA 1  ------------------------------
        var TranslationCasa1 = glm.translate(glm.mat4(1.0),glm.vec3(-10, 33,-7 ));
        
        var ScalingCasa1 = glm.scale(glm.vec3(1.3, 1.3, 1.3));
        
        var RotMatCasa1 = glm.rotate(glm.mat4(1.0),glm.radians(90),glm.vec3(1,0,0));
        //RotMatCasa1 = glm.rotate(glm.mat4(1.0),glm.radians(180),glm.vec3(1,0,0));
        
        var myModelMatrixCasa1 = RotMatCasa1['*'](TranslationCasa1['*'](ScalingCasa1));
        
        this.casa1.setModelTransformation(myModelMatrixCasa1);
        this.casa1.drawit(ViewMatrix,ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2, this.lightL1,this.lightL2);
        
        //------------------------------ TAÇA ------------------------------
        var TranslationTaca = glm.translate(glm.mat4(1.0),glm.vec3(0, 1.5, 0));
        
        var ScalingTaca = glm.scale(glm.vec3(1, 1, 1));
        
        var myModelMatrixTaca = TranslationTaca['*'](ScalingTaca);
        
        this.taca.setModelTransformation(myModelMatrixTaca);
        this.taca.drawit(ViewMatrix,ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2, this.lightL1,this.lightL2);
        
        //------------------------------ HOLOFOTE 1  ------------------------------
        var TranslationHol = glm.translate(glm.mat4(1.0),glm.vec3(-35,0,0));
        
        var ScalingHol = glm.scale(glm.vec3(4,4, 4));
        
        var RotMatHol = glm.rotate(glm.mat4(1.0),glm.radians(90),glm.vec3(0,1,0));
        
        var myModelMatrixHol = RotMatHol['*'](TranslationHol['*'](ScalingHol));
        
        this.holofote.setModelTransformation(myModelMatrixHol);
        this.holofote.drawit(ViewMatrix,ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2, this.lightL1,this.lightL2);
        
        //------------------------------ HOLOFOTE 2  ------------------------------
        var TranslationHol2 = glm.translate(glm.mat4(1.0),glm.vec3(-35,0,0));
        
        var ScalingHol2 = glm.scale(glm.vec3(4,4, 4));
        
        var RotMatHol2 = glm.rotate(glm.mat4(1.0),glm.radians(-90),glm.vec3(0,1,0));
        
        var myModelMatrixHol2 = RotMatHol2['*'](TranslationHol2['*'](ScalingHol2));
        
        this.holofote.setModelTransformation(myModelMatrixHol2);
        this.holofote.drawit(ViewMatrix,ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2, this.lightL1,this.lightL2);
        
        //------------------------------ SUPORTE ECRÃ 1  ------------------------------
        var TranslationSE = glm.translate(glm.mat4(1.0),glm.vec3(-37.3, 10, 0));
        
        var ScalingSE = glm.scale(glm.vec3(1.3, 15.5, 19.5));
        
        var RotMatSE = glm.rotate(glm.mat4(1.0),glm.radians(-60),glm.vec3(0,1,0));
        
        var myModelMatrixSE = RotMatSE['*'](TranslationSE['*'](ScalingSE));
        
        this.Suporte.setModelTransformation(myModelMatrixSE);
        this.Suporte.drawit(ViewMatrix,ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2, this.lightL1,this.lightL2);
        
        var TranslationSE = glm.translate(glm.mat4(1.0),glm.vec3(-37.3, 5, 0));
        
        var ScalingSE = glm.scale(glm.vec3(1.3, 15.5, 6));
        
        var myModelMatrixSE = RotMatSE['*'](TranslationSE['*'](ScalingSE));
        
        this.Suporte.setModelTransformation(myModelMatrixSE);
        this.Suporte.drawit(ViewMatrix,ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2, this.lightL1,this.lightL2);
        
        
    
        //------------------------------ CARRO 1 ------------------------------
        
        //---------------------- MOVIMENTO DO CARRO ----------------------
        this.angRotacao1 += 2;

        if(this.angRotacao1 == 360)
            this.angRotacao1 = 3;
        
        // MATRIZ IDENTIDADE onde se aplicará todas as operações
        var matIdentidade = glm.mat4(1.0);
        
        
        // TRANSLAÇÃO para a origem
        var matTranslation1 = glm.mat4(1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, -this.positionXCar, -this.positionYCar, -this.positionZCar, 1);
        matTranslation1 = matTranslation1['*'](matIdentidade);
        
        // ROTAÇÃO em torno da origem
        var rotacao = glm.angleAxis(glm.radians(this.angRotacao1),glm.vec3(0, 1, 0)); //Define qual o eixo de rotação: eixo dos y e a cada instante mudo o valor do ângulo
        var matRotacao = glm.toMat4(rotacao);
        matRotacao = matRotacao['*'](matTranslation1);
        
        
        this.positionXCar = 0.1*Math.cos(glm.radians(this.angRotacao1*2*Math.PI/30));
        this.positionYCar = 1;
        this.positionZCar = 0.1*Math.sin(glm.radians(this.angRotacao1*2*Math.PI/30));
        
        
        // SEGUNDA TRANSLAÇÃO 
        var matTranslation2 = glm.mat4(1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, this.positionXCar, this.positionYCar, this.positionZCar, 1);
        matTranslation2 = matTranslation2['*'](matRotacao);
        //---------------------- POSIÇÃO DO CARRO ----------------------
        
        
        // TRANSLAÇÃO
        var TranslationCarro1 = glm.translate(glm.mat4(1.0), glm.vec3(-15, 1, -15));
        
        // ROTAÇÃO
        var myRotCarro1 = glm.vec3(1.0,0.0,0.0);
        var RotMatCarro1 = glm.rotate(glm.radians(90),myRotCarro1);
        
        var ScalingCar1 = glm.scale(glm.vec3(0.5,0.5, 0.5));
        
        
        //var myModelMatrixSol = RotMatSol['*'](TranslationSol['*'](ScalingSol));
        

        // var myModelMatrix = TranslationMat['*'](RotMat['*'](ScalingMat));
        var myModelMatrixCarro1 = matTranslation2['*'](TranslationCarro1)['*'](RotMatCarro1)['*'](ScalingCar1);

        // Computar a transformação aplicada a cada vértice do modelo
        //var MVPmatrixCarro1 = ProjectionMatrix['*'](ViewMatrix['*'](myModelMatrixCarro1));
        
        this.carro1.setModelTransformation(myModelMatrixCarro1);
        this.carro1.drawit(ViewMatrix, ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2, this.lightL1,this.lightL2);
    
        
        this.cameraPositionCarro1 = glm.vec3(myModelMatrixCarro1[3][0],myModelMatrixCarro1[3][1]+2,myModelMatrixCarro1[3][2]);
        this.cameraTargetCarro1 = glm.vec3(matTranslation2[3][0] + 2*Math.cos(this.rotateAngleCarro1),
                                           matTranslation2[3][1],
                                           matTranslation2[3][2] + 2*Math.sin(this.rotateAngleCarro1));
        
        //------------------------------ CARRO 2 ------------------------------
        
        //---------------------- MOVIMENTO DO CARRO ----------------------
        this.angRotacao2 += 3;

        if(this.angRotacao2 == 360)
            this.angRotacao2 = 5;
        
        // MATRIZ IDENTIDADE onde se aplicará todas as operações
        var matIdentidade = glm.mat4(1.0);
        
        
        // TRANSLAÇÃO para a origem
        var matTranslation1 = glm.mat4(1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, -this.positionXCar, -this.positionYCar, -this.positionZCar, 1);
        matTranslation1 = matTranslation1['*'](matIdentidade);
        
        // ROTAÇÃO em torno da origem
        var rotacao = glm.angleAxis(glm.radians(this.angRotacao2),glm.vec3(0, 1, 0)); //Define qual o eixo de rotação: eixo dos y e a cada instante mudo o valor do ângulo
        var matRotacao = glm.toMat4(rotacao);
        matRotacao = matRotacao['*'](matTranslation1);
        
        
        this.positionXCar = 0.1*Math.cos(glm.radians(this.angRotacao2*2*Math.PI/30));
        this.positionYCar = 1;
        this.positionZCar = 0.1*Math.sin(glm.radians(this.angRotacao2*2*Math.PI/30));
        
        
        // SEGUNDA TRANSLAÇÃO 
        var matTranslation2 = glm.mat4(1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, this.positionXCar, this.positionYCar, this.positionZCar, 1);
        matTranslation2 = matTranslation2['*'](matRotacao);
        
        //---------------------- POSIÇÃO DO CARRO ----------------------
        
        
        // TRANSLAÇÃO
        var TranslationCarro1 = glm.translate(glm.mat4(1.0), glm.vec3(-18, 1, -18));
        
        // ROTAÇÃO
        var myRotCarro1 = glm.vec3(1.0,0.0,0.0);
        var RotMatCarro1 = glm.rotate(glm.radians(90),myRotCarro1);
        
        var ScalingCar1 = glm.scale(glm.vec3(0.5,0.5, 0.5));
        
        

        // var myModelMatrix = TranslationMat['*'](RotMat['*'](ScalingMat));
        var myModelMatrixCarro1 = matTranslation2['*'](TranslationCarro1)['*'](RotMatCarro1)['*'](ScalingCar1);

        // Computar a transformação aplicada a cada vértice do modelo
        //var MVPmatrixCarro1 = ProjectionMatrix['*'](ViewMatrix['*'](myModelMatrixCarro1));
        
        this.carro2.setModelTransformation(myModelMatrixCarro1);
        this.carro2.drawit(ViewMatrix, ProjectionMatrix, this.NoiteDia, this.lamp1, this.lamp2, this.lightL1,this.lightL2);
        
        //------------------------------ SCREEN ------------------------------
        //------------------------------ LINHA  ------------------------------
        var TranslationScreen = glm.translate(glm.mat4(1.0),glm.vec3(37, 0, 10));
        var RotMatScreen = glm.rotate(glm.radians(-90), glm.vec3(1.0,0.0,0.0));
        var RotMatScreen2 = glm.rotate(glm.radians(120), glm.vec3(0.0,1.0,0.0));
        var ScalingScreen = glm.scale(glm.vec3(1, 18, 14));
        
        var myModelMatrixScreen =  RotMatScreen2['*'](RotMatScreen['*'](TranslationScreen['*'](ScalingScreen)));
        
        
        this.screen2.setModelTransformation(myModelMatrixScreen);
        this.screen2.drawit(ViewMatrix,ProjectionMatrix);
        
        
    }
    
    render(){
        
        // now going to render to the canvas
        
        this.gl.bindFramebuffer(this.gl.FRAMEBUFFER, null);
        this.gl.enable(this.gl.DEPTH_TEST);

        this.gl.viewport(0, 0, this.gl.canvas.width, this.gl.canvas.height);

        if(this.NoiteDia == 1)
            this.gl.clearColor(0.67,0.923,0.99,1.0);  
        else
            this.gl.clearColor(17/255,17/255,29/255,1.0);  
            
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        
        // set view for the user
        // Tell WebGL how to convert from clip space to pixels

        this.drawscene(this.viewM,this.projM, 0);
        
        //--------------------------------
        
        // render to our targetTexture by binding the framebuffer
        this.gl.bindFramebuffer(this.gl.FRAMEBUFFER, this.fb);
        this.gl.framebufferTexture2D(this.gl.FRAMEBUFFER, this.gl.COLOR_ATTACHMENT0, 
                                   this.gl.TEXTURE_2D, this.targetTexture, 0);
        
        // Tell WebGL how to convert from clip space to pixels
        this.gl.viewport(0, 0, this.targetTextureWidth, this.targetTextureHeight);

        if(this.NoiteDia == 1)
            this.gl.clearColor(0.67,0.923,0.99,1.0);  
        else
            this.gl.clearColor(17/255,17/255,29/255,1.0);  
        
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);

        // TRANSFORMAÇÃO DE VISTA/PERSPETIVA
        
        var ViewMatrix2 = glm.lookAt(this.cameraPositionCarro1, this.cameraTargetCarro1, glm.vec3(0,1,0));
        
        // normally the view matrix should be different   
        this.drawscene(ViewMatrix2,this.projM, 1);
        
    }
    
    
}

var appScenario = new myappScenario('myCanvasScenario');

appScenario.run();
</script>
